In [1]:
import pandas as pd
import pickle

morphemes = pd.read_pickle("verb_prefixes_last.pkl")




In [619]:
morphemes.to_pickle("verb_morphemes.pkl")


In [2]:
morphemes

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
5,11.0,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,,,abdiko,vati,0
16,6119.0,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,,,,abono,vati,0
28,19625.0,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,absorbo,vati,0
29,24056.0,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,abstraho,vati,0
43,24070.0,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,,,adapto,vati,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473.0,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,,,,žuž,ati,0
17913,-37005.0,råzpečętati,NaN,v.tr. pf.,unseal,NaN,verb,råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,,,råz,pečęt,ati,0
17914,-37006.0,råzpečętyvati,NaN,v.tr. ipf.,unseal,NaN,verb,råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,,,råz,pečęty,vati,0
17917,-30731.0,smogti,NaN,v.tr. pf.,"manage to, be able to, cope, make it",NaN,verb,?,?,mogti,smogti,smogti,mogti,,,s,mꜵg,ti,0


In [27]:
AO = 'ꜵ'
YI = "⒥"
NASAL = "⒩"
BACKTICK = "’"

In [335]:
morphemes.query('en.str.contains("wax")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
7585,28346.0,navoščiti,NaN,v.tr. pf.,wax,NaN,verb,?,?,ščiti,navoščiti,navošč,ščiti,,,na,vošč,iti,0
9797,29784.0,ovoščiti,NaN,v.tr. pf.,wax,NaN,verb,?,?,ščiti,ovoščiti,ovošč,ščiti,,,o,vošč,iti,0
16659,32609.0,voščiti,NaN,v.tr. ipf.,wax,NaN,verb,?,?,ščiti,voščiti,vošč,ščiti,,,,vošč,iti,0


In [336]:
def split_carefully(x, true_stem):
    if " " in x:
        x = x.split(" ")[0]
    if any(weird in true_stem for weird in "[]()?"):
        splitted = re.split(true_stem, x)
        found_stem = x[len(splitted[0]): -len(splitted[1])]
        assert len(splitted) == 2
        return splitted[0], found_stem, splitted[1]
    x = x.replace('å', 'a')
    true_stem = true_stem.replace('å', 'a')

    if AO in true_stem:
        if true_stem.replace(AO, "a") in x:
            return list(x.partition(true_stem.replace(AO, "a")))
    return x.partition(true_stem.replace(AO, "o"))

def insert_YI(word, signature):

    if " " in word:
        word = word.split(" ")[0]

    if word[-3:] == "ęti":
        word = word[:-3] + "in" + NASAL + "ti"
    if "trim" not in word and "klimat" not in word:
        word = word.replace("idti", "jdti").replace("imati", "jmati")
    if signature == {'s', 't'}:
        if word[-3:] == "sti":
            # rasti -> rastti
            word = word[:-3] + "stti"
        
    base, end = word[:-5], word[-5:]
    
    # jehati
    if signature == {'đ', 'h'}:
        end = end.replace("žđati", "h" + YI + "ati")
    
    if signature == {'ć', 's'}:
        end = end.replace("šćati", "st" + YI + "ati")
    if signature == {'ć', 't'}:
        end = end.replace("šćati", "st" + YI + "ati")
        end = end.replace("ćati", "t" + YI + "ati")
    if signature == {'ć', 'č'}:
        end = end.replace("čiti", "ćiti")
        end = end.replace("čati", "ćati")

    if signature == {'đ', 'd'}:
        end = end.replace("žđati", "zd" + YI + "ati")
        end = end.replace("đati", "d" + YI + "ati")

    # puskati?
    if signature == {'ć', 'k'}:
        end = end.replace("šćati", "sk" + YI + "ati")
    if signature == {"c", "č"}:
        end = end.replace("čati", "c" + YI + "ati")
        end = end.replace("čiti", "c" + YI + "iti")

    if signature == {'k', 'č'}:
        base, end = word[:-6], word[-6:]
        end = end.replace("ščiti", "sk" + YI + "iti")
        end = end.replace("čiti", "k" + YI + "iti").replace("čivati", "k" + YI + "ivati")


    # pisati
    if signature == {'š', 's'}:  
        end = end.replace("šati", "s" + YI + "ati")
        end = end.replace("šiti", "s" + YI + "iti")
        
    if signature == {'š', 'h'}:  
        end = end.replace("šiti", "h" + YI + "iti")
    if signature == {'š'}:
        base, end = word[:-6], word[-6:]
        end = end.replace("šiti", "h" + YI + "iti")
        end = end.replace("šati", "h" + YI + "ati")
        end = end.replace("šivati", "h" + YI + "ivati")
        

    if signature == {'č', 'k'}:
        end = end.replace("čati", "k" + YI + "ati")
        
    if signature == {'ž', 'z'}:
        end = end.replace("žati", "z" + YI + "ati")
    if signature == {'ž', 'g'}:
        base, end = word[:-6], word[-6:]

        end = end.replace("žati", "g" + YI + "ati")
        end = end.replace("žiti", "g" + YI + "iti")
        end = end.replace("živati", "g" + YI + "ivati")

    return base + end

def manual_insert(base_verb, true_stem, last_cons=set(), dry_run=True, from_base_verb=True):
    if from_base_verb:
        g = morphemes.query("base_verb == @base_verb")
    else:
        g = morphemes.query("isv == @base_verb")
    manual_insert_g(g, true_stem, last_cons, dry_run)

def manual_insert_g(g, true_stem, last_cons=set(), dry_run=True):

    for i, row in g.isv.apply(lambda x: split_carefully(insert_YI(x, last_cons), true_stem)).items():
        if any(weird in true_stem for weird in "[]()?"):
            found_stem = row[1]
        else: 
            found_stem = true_stem

        if not dry_run:
            morphemes.loc[i, ['_prefix', '_stem', '_suffix']] = row[0], found_stem, row[2]
        raw_isv = g.loc[i, 'isv']
        #if also_insert_noun:
        #    morphemes.loc[[i], 'base_noun'] = true_stem
        #    print(raw_isv, " <- ", true_stem)

        print(raw_isv, "=", row[0], found_stem, row[2])
        

manual_insert("aklimatizovati", "klimat", dry_run=False)

aklimatizovati = a klimat izovati


In [52]:
manual_insert("tykati", "t[y]?k", dry_run=False)

dotknųti = do tk nųti
dotknųti = do tk nųti
dotykati = do tyk ati
dotykati = do tyk ati
spotknųti = spo tk nųti
spotykati = spo tyk ati
tykati =  tyk ati
tykati =  tyk ati
tykati =  tyk ati
tykati =  tyk ati
zatknųti = za tk nųti
zatknųti = za tk nųti
zatykati = za tyk ati
zatykati = za tyk ati


In [21]:
manual_insert("bezpokojiti", "pokoj", dry_run=False, also_insert_noun=True, from_base_verb=False)

bezpokojiti  <-  pokoj
bezpokojiti = bez pokoj iti
bezpokojiti  <-  pokoj
bezpokojiti = bez pokoj iti


In [14]:
morphemes.loc[130, ['base_noun']] = 'klimat'

In [ ]:
manual_insert("uzdråviti", "zdråv", dry_run=True)

In [ ]:
import json
print(len(words_data))
with open("wiktionary_extended_new.json", "w", encoding="utf8") as f:
    json.dump(words_data, f, ensure_ascii=False)

In [25]:
pvc[pvc < 10]

os          9
p           9
sų          8
råzs        8
odv         8
           ..
odza        1
sp          1
sebe        1
nena        1
izsysati    1
Name: _prefix, Length: 103, dtype: int64

In [28]:
import json

with open("wiktionary_extended_new.json", "r", encoding="utf8") as f:
    words_data = json.load(f)

In [150]:
svc

iti         1330
ati         1215
vati         626
ti           403
jati         331
nųti         218
ěti          207
⒥ati         174
ivati         54
avjati        46
⒩ti           42
aviti         41
jiti          32
avati         30
ovati         27
ěvati         26
              26
vyvati        16
⒥iti          16
yvati         14
ųti           13
oviti         12
evati         12
ějati          9
vti            9
izovati        8
ajati          6
jniti          5
stti           5
jnjati         5
tati           4
⒥ivati         4
niti           3
ějivati        3
in⒩ti          3
njati          2
inųti          2
iveti          1
ęti            1
opiti          1
opivati        1
ropiti         1
ropivati       1
Name: _suffix, dtype: int64

In [173]:
svc

iti         1334
ati         1219
vati         609
ti           403
jati         331
nųti         218
ěti          207
⒥ati         174
ivati         57
avjati        46
⒩ti           42
aviti         41
yvati         31
avati         30
ovati         27
jiti          27
ěvati         26
              18
vyvati        16
⒥iti          16
ųti           13
oviti         12
evati         12
vti            9
ati,           8
izovati        8
ajati          6
ějati          6
jniti          5
jnjati         5
stti           5
tati           4
⒥ivati         4
niti           3
in⒩ti          3
njati          2
inųti          2
iveti          1
ęti            1
opiti          1
opivati        1
ropiti         1
ropivati       1
Name: _suffix, dtype: int64

In [148]:
morphemes['_suffix'] = morphemes['_suffix'].str.replace("յ", YI).str.replace("˛", NASAL)

In [661]:
# morphemes.query("_suffix in @sWTF")

In [628]:
pvc[pvc < 8].head(33)

vy        7
p         7
obez      7
spo       7
razpro    6
prěs      5
nad       5
zane      5
us        5
pr        4
poz       4
pov       4
razs      4
proiz     4
de        4
os        4
zado      4
izna      4
vȯ        4
pripo     4
obȯ       4
ne        3
prěo      3
nas       3
prědpo    3
poza      3
råzpro    3
råzs      2
razo      2
razpo     2
sųpro     2
prone     2
sne       2
Name: _prefix, dtype: int64

In [622]:
svc

iti        1298
ati        1214
vati        600
ti          435
jati        333
nųti        218
ěti         206
⒥ati        184
⒩ti          45
ivati        44
yvati        42
ovati        41
avjati       39
⒥iti         38
aviti        35
avati        30
jiti         27
ěvati        26
             16
vyvati       16
ųti          15
evati        12
oviti        10
vti           9
izovati       8
ati,          8
ajati         6
jnjati        5
jniti         5
⒥ivati        4
ějati         3
niti          3
titi          3
njati         2
inųti         2
veti          1
in⒩ti         1
tati          1
Name: _suffix, dtype: int64

In [624]:
morphemes.query("_suffix == 'in⒩ti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
11065,22659.0,povŕtati,NaN,v.tr. ipf.,turn (tr.),NaN,verb,?,?,vŕnųti,povŕtati,povŕt,?ŕtati?,,,pov,ŕ,tati,1


In [680]:

possible_prefixes = {
    "ne", "bez", "naj",
    "do", "iz", "na", "nad", "ne", "o", "ob", "obez", "od", "po", "pod", 
    "prě", "prěd", "pri", "pro", 
    "råz",
    "de", 
    "s", "sȯ",
    "sų", "u", "v", 
    "vȯz", "vȯ",
    "vy", "za"
}

In [681]:
possible_prefixes - set(morphemes._prefix.unique())

{'naj'}

In [683]:
morphemes.query('_prefix not in @possible_prefixes')._prefix.value_counts()

          2005
råzpro       9
spo          7
råzs         6
zane         5
          ... 
råzȯ         1
odȯ          1
protiv       1
odza         1
nedo         1
Name: _prefix, Length: 74, dtype: int64

In [684]:
morphemes.query('_prefix not in @possible_prefixes')._prefix.unique()

array(['', 'a', 'doråz', 'doz', 'izna', 'izne', 'izȯ', 'izu', 'mråzo',
       'nadu', 'napo', 'nas', 'nedoråz', 'nedo', 'nena', 'nezado', 'obȯ',
       'odȯ', 'odpo', 'odza', 'opo', 'os', 'podȯ', 'podråz', 'poråz',
       'posȯ', 'pos', 'pov', 'poza', 'poz', 'prědȯ', 'prědpo', 'prědpri',
       'prěds', 'prěna', 'prěo', 'prěpo', 'prěråz', 'prěs', 'prěvȯz',
       'prěvȯzȯ', 'prina', 'pripo', 'pr', 'proiz', 'prone', 'protiv',
       'råzo', 'råzȯ', 'råzpo', 'råzpro', 'råzråz', 'råzråznjati', 'råzs',
       'rųko', 'samou', 'sde', 'sebe', 'sna', 'sne', 'spolu', 'spo',
       'sråz', 'sųpro', 'us', 'vȯzȯ', 'vȯzpo', 'vz', 'zado', 'zaj',
       'zane', 'zapo', 'zapro', 'zau'], dtype=object)

In [ ]:
¬ne¬prě√měn^n|y

In [656]:
svc = morphemes._suffix.value_counts()
pvc = morphemes._prefix.value_counts()


sWTF = svc[svc < 6].index
pWTF = pvc[pvc < 6].index


morphemes.query("_prefix in @pWTF or _suffix in @sWTF or _suffix == ''")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
130,24156.0,aklimatizovati,NaN,v.intr. ipf.,acclimatize,I,verb,?,?,aklimatizovati,aklimatizovati,aklimatiz,aklimatizovati,|aklimatizacija,klimat,a,klimat,izovati,0
784,36698.0,bezpokojiti,NaN,v.refl. ipf.,"worry, be anxious, feel uneasy",NaN,verb,?,?,†pokojiti,bezpokojiti sę,bezpoko,?kojiti?,,pokoj,bez,pokoj,iti,0
785,19654.0,bezpokojiti,NaN,v.tr. ipf.,"disturb, trouble, upset, bother",NaN,verb,?,?,†pokojiti,bezpokojiti,bezpoko,?kojiti?,,pokoj,bez,pokoj,iti,0
1185,22979.0,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,,,,,0
1717,25153.0,deblokovati,NaN,v.intr. ipf./pf.,unblock,I,verb,de’,blokovati,blokovati,deblokovati,deblok,blokovati,,,de,bloko,vati,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,35852.0,zanepokojiti,NaN,v.tr. pf.,worry,NaN,verb,?,?,†pokojiti,zanepokojiti,zanepoko,nepokojiti,,,zane,pokoj,iti,0
17397,21659.0,zapoznati,NaN,v.tr. pf.,acquaint,NaN,verb,?,?,znati,zapoznati,zapozn,znati,,,zapo,zn,ati,0
17407,21238.0,zapropastiti,NaN,v.refl. pf.,disappear completely,NaN,verb,?,?,pasti,zapropastiti sę,zapropast,?pastiti?,,,zapro,pas,titi,0
17539,23977.0,zauvažati,NaN,v.tr. ipf.,notice,NaN,verb,?,?,važiti,zauvažati,zauvaž,uvažati,,,zau,vag,⒥ati,0


In [226]:
pWTF

Index(['prěs', 'nad', 'zane', 'us', 'råzpros', 'proiz', 'pov', 'poz', 'pr',
       'razs', 'obȯ', 'pripo', 'de', 'zado', 'izra', 'vȯ', 'ne', 'vȯzs', 'ra',
       'prěo', 'prědpo', 'råzpro', 'sra', 'obez', 'pora', 'ura', 'razpro',
       'sna', 'razpo', 'razo', 'sųpro', 'prone', 'vȯzpo', 'spolu', 'sne',
       'råzv', 'prěds', 'obu', 'izna', 'obezhra', 'obe', 'obs', 'naz', 'napo',
       'nadu', 'mråzo', 'poza', 'podraz', 'doz', 'prědv', 'prěna', 'prěpo',
       'bez', 'prěråz', 'nezado', 'izne', 'nedo', 'izȯ', 'vȯzȯ', 'izu', 'dora',
       'zaj', 'nedora', 'prina', 'nena', 'prěvȯz', 'sebe', 'sde', 'odȯ',
       'samou', 'rųko', 'odza', 'opo', 'pos', 'prědȯ', 'prědpri', 'razȯ', 'a',
       'protiv', 'zapo'],
      dtype='object')

In [403]:
check_if_orphan("vrěti"), "vrěti" in morphemes.base_verb.unique()

(False, True)

In [563]:
morphemes.query("base_verb == 'prisposobiti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
11832,18491.0,prisposobiti,NaN,v.tr. pf.,"customise, customize, adapt, adjust (tr.)",NaN,verb,?,?,prisposobiti,prisposobiti,prisposob,biti,,,pri,sposꜵb,iti,0
11833,18493.0,prisposobjati,NaN,v.tr. ipf.,"customise, customize, adapt, adjust (tr.)",NaN,verb,?,?,prisposobiti,prisposobjati,prisposob,?bjati?,,,pri,sposꜵb,jati,0
15878,3830.0,usposabjati,NaN,v.tr. ipf.,"qualify, prepare (for a task), fit out, render...",NaN,verb,?,?,prisposobiti,usposabjati,usposab,?abjati?,,,u,sposꜵb,jati,0
15879,3831.0,usposobiti,NaN,v.tr. pf.,"qualify, prepare (for a task), fit out, render...",NaN,verb,?,?,prisposobiti,usposobiti,usposob,biti,,,u,sposꜵb,iti,0


In [562]:
check_if_orphan('prisposobiti')

True

In [633]:
PREV = set(ORPHANS) or set(morphemes.base_verb.unique())
PREV = PREV & set(morphemes.base_verb.unique())

ORPHANS = set()

for base in PREV:
    if check_if_orphan(base):
        print(base, check_if_orphan(base))
        ORPHANS |= {base}



råzmetati True
naklåti True


In [634]:
ORPHANS -= {'†ložiti', '†niziti', '†pokojiti', '†vysiti',
 '†gynųti',
 '†jati',
 '†kladati',
 '†lagati',
 '†městiti',
 '†nažiti',
 '†niknųti',
 '†pitati',
 '†plåšiti',
 '†spěti',
 '†strěti',
 '†trčati',
 '†vihnųti',
  'končati', 'jęti', 'slåditi', 'věděti', 'poriti', 'siliti', 'sȯmknųti', 'skråtiti', 'vladati', 'imati',
           "aklimatizovati", 'belěti', 'mråzosušati', 'bočiti', 'programovati', 'projektovati', 
            'obuzdati', 'opravdati', 'odšlupati', "ogråditi", "råzgråditi", "podobniti", 'prisposobiti',
            'dokonati', 'drěmnųti', 'nagråditi', 'zamknųti', 'utočniti', 'trojiti', 'niti', 'sȯvětovati', 'kati', 'obuti', 'odčajati',
            #?
            'vŕnųti', 'mrěti', 'sųsrědotočati', 'sȯniti', 'sloviti', 'občiti', 'pok' + YI + 'iti', #''
           }

morphemes.query("base_verb in @ORPHANS").shape, len(ORPHANS)

((3, 19), 2)

In [637]:
morphemes.query("base_verb in @ORPHANS").sort_values(by='base_verb')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
7237,32013.0,nakalati,NaN,v.tr. ipf.,pin onto,NaN,verb,?,?,naklåti,nakalati,nakal,?kalati?,,,na,kal,ati,0


In [665]:
morphemes.query("isv.str.contains('å') and not (_prefix.str.contains('å') or _stem.str.contains('å')) and not _prefix.str.contains('raz')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
1248,10343.0,byti ostråžny,NaN,v.ipf.,"look out, watch out",NaN,verb,?,?,byti,byti ostråžny,byti ostråžny,?byti ostråžny?,,,,by,ti,0
1250,23728.0,byti prěhlåđeny,NaN,v.ipf.,have a cold,NaN,verb,?,?,byti,byti prěhlåđeny,byti prěhlåđeny,?byti prěhlåđeny?,,,,by,ti,0
1252,4190.0,byti sȯglåsny,NaN,v.ipf.,agree,NaN,verb,?,?,byti,byti sȯglåsny,byti sȯglåsny,?byti sȯglåsny?,,,,by,ti,0
1691,6973.0,dati råbotų,#(da),v.pf.,"hire, employ",NaN,verb,?,?,dati,dati råbotų,dati råbotų,?dati råbotų?,,,,da,ti,0
1700,6979.0,davati råbotų,NaN,v.ipf.,"hire, employ",NaN,verb,?,?,davati,davati råbotų,davati råbotų,?davati råbotų?,,,,dav,ati,-1
2383,6185.0,dŕžati glådovkų,#(dŕži),v.ipf.,hunger-strike,NaN,verb,?,?,dŕžati,dŕžati glådovkų,dŕžati glådovkų,?dŕžati glådovkų?,,,,dŕž,ati,0
6241,5729.0,mahati glåvojų,NaN,v.ipf.,shake one’s head,NaN,verb,?,?,mahati,mahati glåvojų,mahati glåvojų,?mahati glåvojų?,,,,mah,ati,0
6245,5730.0,mahnųti glåvojų,NaN,v.pf.,shake one’s head,NaN,verb,?,?,mahati,mahnųti glåvojų,mahnųti glåvojų,?mahnųti glåvojų?,,,,mah,nųti,0
16483,569.0,"vladati, vlåděti",(+5),v.intr. ipf.,"reign, rule",NaN,verb,?,?,vladati,"vladati, vlåděti","vladati, vlåd","?ladati, vlåděti?",,,,vlad,"ati,",0
16484,13670.0,"vladati, vlåděti",(+5),v.intr. ipf.,"govern, control, manage, be in control of",NaN,verb,?,?,vladati,"vladati, vlåděti","vladati, vlåd","?ladati, vlåděti?",,,,vlad,"ati,",0


In [674]:
morphemes.query("_prefix.str.contains('raz')")._prefix.value_counts()

Series([], Name: _prefix, dtype: int64)

In [902]:
morphemes._prefix = morphemes._prefix.str.replace('raz', 'råz')
morphemes._prefix = morphemes._prefix.str.replace('blago', 'blågo')

In [689]:
BV_IDX = morphemes.query("_prefix == 'pr'").sort_values(by='base_verb').index


In [701]:
morphemes.loc[BV_IDX]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
12009,18512.0,proglašati,NaN,v.tr. ipf.,proclaim,NaN,verb,?,?,glåsiti,proglašati,proglaš,oglašati,,,pro,glas,⒥ati,0
12159,5784.0,prosvětiti,NaN,v.tr. pf.,#enlighten,NaN,verb,?,?,světiti,prosvětiti,prosvět,osvětiti,,,pro,svět,iti,0
12156,5783.0,prosvěćati,NaN,v.tr. ipf.,#enlighten,NaN,verb,?,?,světiti,prosvěćati,prosvěć,osvěćati,,,pro,svět,⒥ati,0
11972,23911.0,pročišćati,NaN,v.tr. ipf.,"cleanse, clean thoroughly",NaN,verb,?,?,čistiti,pročišćati,pročišć,očišćati,,,pro,čist,⒥ati,0


In [690]:

BV = morphemes.query("_prefix == 'pr'").base_verb.unique()

EN_BV = morphemes.query("base_verb in @BV").en.unique()
morphemes.query("en in @EN_BV")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8691,29030.0,očišćati,NaN,v.tr. ipf.,"cleanse, purify",NaN,verb,?,?,očišćati,očišćati,očišć,?čišćati?,,,,očišć,ati,0
9148,173.0,oglåsiti,NaN,v.tr. pf.,"announce, declare, herald",NaN,verb,?,?,glåsiti,oglåsiti,oglås,glåsiti,,,o,glås,iti,0
9149,1481.0,oglašati,NaN,v.tr. ipf.,"announce, declare, herald",NaN,verb,?,?,oglašati,oglašati,oglaš,?glašati?,,,,oglaš,ati,0
9696,29721.0,osvěćati,NaN,v.tr. ipf.,illuminate,NaN,verb,?,?,osvěćati,osvěćati,osvěć,?ěćati?,,,,osvěć,ati,0
9702,29724.0,osvětiti,NaN,v.tr. pf.,illuminate,NaN,verb,?,?,osvětiti,osvětiti,osvět,?ětiti?,,,,osvět,iti,0
11971,23912.0,pročistiti,NaN,v.tr. pf.,"cleanse, clean thoroughly",NaN,verb,?,?,čistiti,pročistiti,pročist,čistiti,,,pro,čist,iti,0
11972,23911.0,pročišćati,NaN,v.tr. ipf.,"cleanse, clean thoroughly",NaN,verb,?,?,očišćati,pročišćati,pročišć,očišćati,,,pr,očišć,ati,0
12008,18514.0,proglåsiti,NaN,v.tr. pf.,proclaim,NaN,verb,?,?,glåsiti,proglåsiti,proglås,glåsiti,,,pro,glås,iti,0
12009,18512.0,proglašati,NaN,v.tr. ipf.,proclaim,NaN,verb,?,?,oglašati,proglašati,proglaš,oglašati,,,pr,oglaš,ati,0
12156,5783.0,prosvěćati,NaN,v.tr. ipf.,#enlighten,NaN,verb,?,?,osvěćati,prosvěćati,prosvěć,osvěćati,,,pr,osvěć,ati,0


In [691]:
BV

array(['očišćati', 'oglašati', 'osvěćati', 'osvětiti'], dtype=object)

In [693]:
g = morphemes.query("base_verb == 'očišćati'")
manual_insert_g(g, "čist", last_cons={"t", "ć"}, dry_run=0)
set_base_verb(g, 'čistiti')


očišćati = o čist ⒥ati
pročišćati = pro čist ⒥ati


In [699]:
morphemes.query("isv.str.contains('svěć')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
9696,29721.0,osvěćati,NaN,v.tr. ipf.,illuminate,NaN,verb,?,?,světiti,osvěćati,osvěć,?ěćati?,,,o,svět,⒥ati,0
12156,5783.0,prosvěćati,NaN,v.tr. ipf.,#enlighten,NaN,verb,?,?,světiti,prosvěćati,prosvěć,osvěćati,,,pro,svět,⒥ati,0
17493,21680.0,zasvěćati,NaN,v.intr. ipf.,"brighten, light up",NaN,verb,?,?,zasvětiti,zasvěćati,zasvěć,?ěćati?,,,,zasvět,⒥ati,0


In [700]:

g = morphemes.query("isv.str.contains('svě[ćt]')")
manual_insert_g(g, "svět", last_cons={"t", "ć"}, dry_run=0)
set_base_verb(g, 'světiti')

obsvětliti = ob svět liti
obsvětljati = ob svět ljati
osvěćati = o svět ⒥ati
osvětiti = o svět iti
osvětliti = o svět liti
osvětljati = o svět ljati
prosvěćati = pro svět ⒥ati
prosvětiti = pro svět iti
råzsvětliti = raz svět liti
råzsvětljati = raz svět ljati
zasvěćati = za svět ⒥ati
zasvětiti = za svět iti


In [694]:

g = morphemes.query("base_verb == 'oglašati'")
manual_insert_g(g, "glas", last_cons={"s", "š"}, dry_run=0)
set_base_verb(g, 'glåsiti')

g = morphemes.query("base_verb == 'osvěćati'")
manual_insert_g(g, "svět", last_cons={"t", "ć"}, dry_run=0)
set_base_verb(g, 'světiti')

g = morphemes.query("base_verb == 'osvětiti'")
manual_insert_g(g, "svět", dry_run=0)
set_base_verb(g, 'světiti')


oglašati = o glas ⒥ati
proglašati = pro glas ⒥ati
osvěćati = o svět ⒥ati
prosvěćati = pro svět ⒥ati
osvětiti = o svět iti
prosvětiti = pro svět iti


In [645]:
BV = morphemes.query("_prefix == 'p'").base_verb.unique()

EN_BV = morphemes.query("base_verb in @BV").en.unique()


morphemes.query("en in @EN_BV").sort_values(by='en')


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8388,22524.0,oblěkati,NaN,v.tr. ipf.,"clothe, dress",NaN,verb,?,?,vlěkti,oblěkati,oblěk,?lěkati?,,,ob,lěk,ati,0
8390,22520.0,oblěkti,NaN,v.tr. pf.,"clothe, dress",NaN,verb,?,?,vlěkti,oblěkti,oblěkti,?lěkti?,,,ob,lěk,ti,0
8768,16840.0,oděti,(oděne),v.tr. pf.,"clothe, dress",NaN,verb,?,?,děti,oděti,od,děti,,,o,dě,ti,-1
8770,16838.0,oděvati,NaN,v.tr. ipf.,"clothe, dress",NaN,verb,?,?,oděvati,oděvati,oděv,?ěvati?,,,,od,ěvati,0
8777,5382.0,odgovarjati,NaN,v.tr. ipf.,"answer, reply",NaN,verb,?,?,odgovarjati,odgovarjati,odgovar,?govarjati?,,,,odgovar,jati,0
8778,34094.0,odgovarjati,NaN,v.tr. ipf.,dissuade,NaN,verb,?,?,odgovarjati,odgovarjati,odgovar,?govarjati?,,,,odgovar,jati,0
8780,3242.0,odgovoriti,NaN,v.tr. pf.,"answer, reply",NaN,verb,?,?,govoriti,odgovoriti,odgovor,govoriti,,,od,govꜵr,iti,0
8781,29087.0,odgovoriti,NaN,v.tr. pf.,dissuade,NaN,verb,?,?,govoriti,odgovoriti,odgovor,govoriti,,,od,govꜵr,iti,0
8816,2225.0,odkladati,NaN,v.tr. ipf.,"suspend, postpone, delay",NaN,verb,?,?,odkladati,odkladati,odklad,?kladati?,,,,odklad,ati,0
8817,5568.0,odkladati,NaN,v.tr. ipf.,"put away, put off, put aside, put back",NaN,verb,?,?,odkladati,odkladati,odklad,?kladati?,,,,odklad,ati,0


In [650]:
BV_IDX = morphemes.query("base_verb in @BV").sort_values(by='base_verb').index


In [653]:


g = morphemes.query("base_verb == 'odgovarjati'")
manual_insert_g(g, "govar", dry_run=0)
set_base_verb(g, 'govoriti')

g = morphemes.query("base_verb == 'odpovědati'")
manual_insert_g(g, "věd", dry_run=0)
set_base_verb(g, 'věděti')

g = morphemes.query("base_verb == 'odložiti'")
manual_insert_g(g, "lož", dry_run=0)
set_base_verb(g, '†ložiti')

g = morphemes.query("base_verb == 'oděvati'")
manual_insert_g(g, "dě", dry_run=0)
set_base_verb(g, 'děti')

g = morphemes.query("base_verb == 'otręsati'")
manual_insert_g(g, "tręs", dry_run=0)
set_base_verb(g, 'tręsti')

g = morphemes.query("base_verb == 'osvęćati'")
manual_insert_g(g, "svęt", last_cons={"t", "ć"}, dry_run=0)
set_base_verb(g, 'svętiti')

g = morphemes.query("base_verb == 'odkladati'")
manual_insert_g(g, "klad", dry_run=0)
set_base_verb(g, 'klasti')


odgovarjati = od govar jati
odgovarjati = od govar jati
podgovarjati = pod govar jati
odpovědati = odpo věd ati
odpovědati = odpo věd ati
odpověděti = odpo věd ěti
odložiti = od lož iti
odložiti = od lož iti
podložiti = pod lož iti
oděvati = o dě vati
poděvati = po dě vati
prěoděvati = prěo dě vati
otręsati = o tręs ati
potręsati = po tręs ati
osvęćati = o svęt ⒥ati
posvęćati = po svęt ⒥ati
odkladati = od klad ati
odkladati = od klad ati
podkladati = pod klad ati


In [654]:
morphemes.loc[BV_IDX]


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8777,5382.0,odgovarjati,NaN,v.tr. ipf.,"answer, reply",NaN,verb,?,?,govoriti,odgovarjati,odgovar,?govarjati?,,,od,govar,jati,0
8778,34094.0,odgovarjati,NaN,v.tr. ipf.,dissuade,NaN,verb,?,?,govoriti,odgovarjati,odgovar,?govarjati?,,,od,govar,jati,0
10362,18254.0,podgovarjati,NaN,v.tr. ipf.,incite,NaN,verb,?,?,govoriti,podgovarjati,podgovar,odgovarjati,,,pod,govar,jati,0
8816,2225.0,odkladati,NaN,v.tr. ipf.,"suspend, postpone, delay",NaN,verb,?,?,klasti,odkladati,odklad,?kladati?,,,od,klad,ati,0
8817,5568.0,odkladati,NaN,v.tr. ipf.,"put away, put off, put aside, put back",NaN,verb,?,?,klasti,odkladati,odklad,?kladati?,,,od,klad,ati,0
10376,32682.0,podkladati,NaN,v.tr. ipf.,put under,NaN,verb,?,?,klasti,podkladati,podklad,odkladati,,,pod,klad,ati,0
8865,2984.0,odložiti,NaN,v.tr. pf.,"suspend, postpone, delay",NaN,verb,?,?,†ložiti,odložiti,odlož,?ložiti?,,,od,lož,iti,0
8866,5569.0,odložiti,NaN,v.tr. pf.,"put away, put off, put aside, put back",NaN,verb,?,?,†ložiti,odložiti,odlož,?ložiti?,,,od,lož,iti,0
10402,32684.0,podložiti,NaN,v.tr. pf.,put under,NaN,verb,?,?,†ložiti,podložiti,podlož,odložiti,|podložka,,pod,lož,iti,0
8770,16838.0,oděvati,NaN,v.tr. ipf.,"clothe, dress",NaN,verb,?,?,děti,oděvati,oděv,?ěvati?,,,o,dě,vati,0


In [ ]:

morphemes.query("isv.str.contains('blågo')").isv.str.partition('blågo')[2].apply(lambda x: x + str(x in morphemes.isv.unique() or x in morphemes.base_verb.unique()))


In [899]:
morphemes.query("isv.str.contains('blågo')").isv.str.partition('blågo')[2].apply(lambda x: x + str(x in morphemes.isv.unique() or x in morphemes.base_verb.unique()))


887         daritiTrue
899      slavjatiFalse
900        slovitiTrue
907         volětiTrue
912         želatiTrue
10292       daritiTrue
10293       želatiTrue
Name: 2, dtype: object

In [897]:
#morphemes.loc[12828]

In [900]:


g = morphemes.query("base_verb == 'blågodariti'")
manual_insert_g(g, "dar", dry_run=0)
set_base_verb(g, 'dariti')

g = morphemes.query("base_verb == 'blågoslavjati'")
manual_insert_g(g, "slav", dry_run=0)
set_base_verb(g, 'sloviti')

g = morphemes.query("base_verb == 'blågosloviti'")
manual_insert_g(g, "slov", dry_run=0)
set_base_verb(g, 'sloviti')

g = morphemes.query("base_verb == 'blågovolěti'")
manual_insert_g(g, "vol", dry_run=0)
set_base_verb(g, 'volěti')

g = morphemes.query("base_verb == 'blågoželati'")
manual_insert_g(g, "žel", dry_run=0)
set_base_verb(g, 'želati')


blågodariti = blago dar iti
poblågodariti = poblago dar iti
blågoslavjati = blago slav jati
blågosloviti = blago slov iti
blågovolěti = blago vol ěti
blågoželati = blago žel ati
poblågoželati = poblago žel ati


In [642]:
morphemes.query("base_verb == 'odbirati' or base_verb == 'brati'")

g = morphemes.query("base_verb == 'odbirati'")
manual_insert_g(g, "bir", dry_run=0)
set_base_verb(g, 'brati')
morphemes.loc[g.index]

odbirati = od bir ati
odbirati = od bir ati
podbirati = pod bir ati


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8710,19937.0,odbirati,NaN,v.tr. ipf.,take away,NaN,verb,?,?,brati,odbirati,odbir,?birati?,,,od,bir,ati,0
8711,29045.0,odbirati,NaN,v.tr. ipf.,select,NaN,verb,?,?,brati,odbirati,odbir,?birati?,,,od,bir,ati,0
10333,35887.0,podbirati,NaN,v.tr. ipf.,pick up,NaN,verb,?,?,brati,podbirati,podbir,odbirati,,,pod,bir,ati,0


In [652]:
morphemes.query("base_verb == 'svętiti' or base_verb == 'klasti' or base_verb == 'krasti'")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4925,901.0,klasti,(klade),v.tr. ipf.,"put, lay",NaN,verb,?,?,klasti,klasti,klasti,klasti,,,,klas,ti,0
5472,26804.0,krasti,(krade),v.refl. ipf.,sneak,NaN,verb,?,?,krasti,krasti sę,krasti,krasti,,,,kras,ti,0
5473,627.0,krasti,(krade),v.tr. ipf.,steal,NaN,verb,?,?,krasti,krasti,krasti,krasti,,,,kras,ti,0
8366,28779.0,obkradati,NaN,v.tr. ipf.,rob,NaN,verb,?,?,krasti,obkradati,obkrad,?kradati?,,,ob,krad,ati,1
8367,33740.0,obkrasti,(obkrade),v.tr. pf.,rob,NaN,verb,?,?,krasti,obkrasti,obkrasti,krasti,,,ob,kras,ti,0
9701,29723.0,osvętiti,NaN,v.tr. pf.,"sanctify, hallow, consecrate",NaN,verb,?,?,svętiti,osvętiti,osvęt,svętiti,,,o,svęt,iti,0
10968,652.0,posvętiti,NaN,v.tr. pf.,"dedicate, devote",NaN,verb,?,?,svętiti,posvętiti,posvęt,svętiti,,,po,svęt,iti,0
14756,1440.0,svętiti,NaN,v.tr. ipf.,hallow,NaN,verb,?,?,svętiti,svętiti,svęt,?ętiti?,,,,svęt,iti,0
15671,941.0,ukrasti,(ukrade),v.tr. pf.,steal,NaN,verb,?,?,krasti,ukrasti,ukrasti,krasti,,,u,kras,ti,0


In [636]:
set_base_verb(morphemes.query("base_verb == 'råzmetati'"), 'metati')


In [614]:
set_base_verb(morphemes.query("base_verb.str.contains('mlå[dđ]')"), 'mlåděti')


In [615]:
# morphemes.query("base_verb.str.contains('((na)|(o))(žiti)|(živati)') and base_verb != '†ložiti'")

morphemes.query("base_verb == 'žiti'")
manual_insert('žiti', 'ži', dry_run=0)

dožiti = do ži ti
doživeti = do ži veti
nadužiti = nadu ži ti
naduživati = nadu ži vati
ožiti = o ži ti
oživati = o ži vati
prěžiti = prě ži ti
prěžiti = prě ži ti
prěžiti = prě ži ti
prěživati = prě ži vati
prěživati = prě ži vati
proživati = pro ži vati
užiti = u ži ti
uživati = u ži vati
vžiti = v ži ti
vživati = v ži vati
žiti =  ži ti
žiti =  ži ti


In [553]:
morphemes.loc[12242, 'base_verb'] = 'žiti'

In [618]:
manual_insert("slizgnųti", 'slizg', dry_run=0)
manual_insert("stignųti", 'stig', dry_run=0)
manual_insert("svatiti", 'svat', dry_run=0)
manual_insert("råzmetati", 'met', dry_run=0)
manual_insert("obråtiti", 'obrå', dry_run=0)
manual_insert("naklåti", 'kal', dry_run=0)



poslizgnųti = po slizg nųti
nastignųti = na stig nųti
osvatiti = o svat iti
råzmetati = raz met ati
råzmetyvati = raz met yvati
prěobråtiti = prě obrå titi
prěobråtiti = prě obrå titi
nakalati = na kal ati


In [611]:
g = morphemes.query("en.str.contains('perspire')")
g
manual_insert_g(g, "pot", dry_run=0)
set_base_verb(g, 'potiti')
#morphemes.loc[g.index]

potěti =  pot ěti
potiti =  pot iti
spotěti = s pot ěti
spotiti = s pot iti


In [609]:
g = morphemes.query("isv.str.contains('odȯjdti')")
manual_insert_g(g, "jd", dry_run=0)
set_base_verb(g, 'idti')
morphemes.loc[g.index]

odȯjdti = odȯ jd ti
podȯjdti (k) = podȯ jd ti
podȯjdti = podȯ jd ti


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8892,181.0,odȯjdti,(odȯjde; odšel),v.intr. pf.,"go away, walk away, leave, backtrack",NaN,verb,?,?,idti,odȯjdti,odȯjdti,?ȯjdti?,,,odȯ,jd,ti,0
10426,34766.0,podȯjdti (k),(podȯjde; podšėl) (+3),v.intr. pf.,approach,NaN,verb,?,?,idti,podȯjdti (k),podȯjdti (k),?ȯjdti (k)?,,,podȯ,jd,ti,0
10427,36534.0,podȯjdti,(podȯjde; podšėl),v.intr. pf.,"suit, be suitable, befit, go with",NaN,verb,?,?,idti,podȯjdti,podȯjdti,odȯjdti,,,podȯ,jd,ti,0


In [604]:
#
set_base_verb(morphemes.query("base_verb == 'prědviděti'"), 'viděti')


manual_insert('viděti', 'vid', dry_run=0)

nenaviděti = nena vid ěti
prědviděti = prěd vid ěti
prědvidyvati = prěd vid yvati
uviděti = u vid ěti
zaviděti = za vid ěti


In [599]:
set_base_verb(morphemes.query("base_verb == 'råzvědati'"), 'věděti')
morphemes.query("base_verb == 'věděti'")

manual_insert_g(morphemes.query("derived_nouns.str.contains('råzvědka')"), 'věd', dry_run=0)

råzvědati = raz věd ati
råzvědyvati = raz věd yvati


In [583]:
set_base_verb(morphemes.query("base_verb == 'propasti'"), 'pasti')

manual_insert('pasti', 'pas', dry_run=0)

izpasti = iz pas ti
napasti = na pas ti
napasti = na pas ti
napasti = na pas ti
odpasti = od pas ti
opasti = o pas ti
pasti =  pas ti
pasti =  pas ti
pasti =  pas ti
pripasti = pri pas ti
propasti = pro pas ti
propasti = pro pas ti
propasti = pro pas ti
råzpasti = raz pas ti
spasati = s pas ati
spasti = s pas ti
spasti = s pas ti
upasti = u pas ti
vpasti = v pas ti
zapasti = za pas ti
zapropastiti = zapro pas titi


In [592]:
morphemes.loc[[8939, 8941], 'base_verb'] = 'pok' + YI + 'iti'

In [593]:

manual_insert('počęti', 'počin', dry_run=0)
morphemes.query("base_verb.str.contains('počęti')")


počęti sę =  počin ⒩ti
počęti =  počin ⒩ti
počinati =  počin ati
počinati =  počin ati
započęti = za počin ⒩ti
započinati = za počin ati


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
10306,22788.0,počęti sę,(počne),v.tr. pf.,"begin (intr.), start (intr.)",NaN,verb,?,?,počęti,počęti sę,počęti,?čęti sę?,,,,počin,⒩ti,-1
10307,1078.0,počęti,(počne),v.tr. pf.,"begin (tr.), start (tr.), commence",NaN,verb,?,?,počęti,počęti,počęti,?čęti?,,,,počin,⒩ti,-1
10315,10675.0,počinati,NaN,v.refl. ipf.,"begin (intr.), start (intr.)",NaN,verb,?,?,počęti,počinati sę,počin,?činati?,,počin,,počin,ati,0
10316,2835.0,počinati,NaN,v.tr. ipf.,"begin (tr.), start (tr.), commence",NaN,verb,?,?,počęti,počinati,počin,?činati?,,počin,,počin,ati,0
17386,32371.0,započęti,(započne),v.tr. pf.,"initiate, commence, kick off, originate, start up",NaN,verb,?,?,počęti,započęti,započęti,počęti,,,za,počin,⒩ti,-1
17387,19401.0,započinati,NaN,v.tr. ipf.,"initiate, commence, kick off, originate, start up",NaN,verb,?,?,počęti,započinati,započin,počinati,,,za,počin,ati,0


In [632]:
morphemes.query("base_verb.str.contains('poč') or base_verb.str.contains('pok⒥iti')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8939,5605.0,odpočęti,(odpočne),v.intr. pf.,"rest, relax",NaN,verb,?,?,pok⒥iti,odpočęti,odpočęti,počęti,,,od,poč,in⒩ti,-1
8941,3295.0,odpočivati,NaN,v.intr. ipf.,"rest, relax",NaN,verb,?,?,pok⒥iti,odpočivati,odpoč,?čivati?,,,od,poč,ivati,-1
10306,22788.0,počęti sę,(počne),v.tr. pf.,"begin (intr.), start (intr.)",NaN,verb,?,?,počęti,počęti sę,počęti,?čęti sę?,,,,počin,⒩ti,-1
10307,1078.0,počęti,(počne),v.tr. pf.,"begin (tr.), start (tr.), commence",NaN,verb,?,?,počęti,počęti,počęti,?čęti?,,,,počin,⒩ti,-1
10315,10675.0,počinati,NaN,v.refl. ipf.,"begin (intr.), start (intr.)",NaN,verb,?,?,počęti,počinati sę,počin,?činati?,,počin,,počin,ati,0
10316,2835.0,počinati,NaN,v.tr. ipf.,"begin (tr.), start (tr.), commence",NaN,verb,?,?,počęti,počinati,počin,?činati?,,počin,,počin,ati,0
17386,32371.0,započęti,(započne),v.tr. pf.,"initiate, commence, kick off, originate, start up",NaN,verb,?,?,počęti,započęti,započęti,počęti,,,za,počin,⒩ti,-1
17387,19401.0,započinati,NaN,v.tr. ipf.,"initiate, commence, kick off, originate, start up",NaN,verb,?,?,počęti,započinati,započin,počinati,,,za,počin,ati,0


In [586]:
morphemes.loc[17387, ['base_verb', '_prefix','_stem','_suffix']] = ['počęti', 'za', 'poč', 'inati']
morphemes.loc[[10315, 10316], ['base_verb']] = ['počęti']

In [ ]:
morphemes.query("base_verb == 'pasti'")

manual_insert('počinati', 'ni', dry_run=1)

In [575]:
set_base_verb(morphemes.query('base_verb in ["råzniti", "niti"]'), 'niti')
manual_insert('niti', 'ni', dry_run=0)

råzniti = raz ni ti
råzråzniti = razraz ni ti
råzraznjati = razraznjati ni 


In [894]:
manual_insert('niti', 'ni?', dry_run=0)



råzniti = råz ni ti
råzråzniti = råzråz ni ti
råzraznjati = råzraz n jati


In [896]:
morphemes.loc[
    morphemes[morphemes.isv == 'råzraznjati'].index,
    '_prefix'
] = "råzråz"


In [568]:
set_base_verb(morphemes.query('base_verb == "proslaviti"'), 'slaviti')

In [566]:
manual_insert('sloviti', 'sl[oa]v', dry_run=0)
manual_insert('obuzdati', 'uzd', dry_run=0)
manual_insert('odšlupati', 'šlup', dry_run=0)
manual_insert('opravdati', 'pravd', dry_run=0)
manual_insert('proslaviti', 'slav', dry_run=0)



izslavjati = iz slav jati
izslavjati = iz slav jati
izsloviti = iz slov iti
izsloviti = iz slov iti
obuzdati = ob uzd ati
obuzdyvati = ob uzd yvati
odšlupati = od šlup ati
odšlupyvati = od šlup yvati
opravdati = o pravd ati
opravdati = o pravd ati
opravdyvati = o pravd yvati
opravdyvati = o pravd yvati
proslaviti = pro slav iti
proslaviti = pro slav iti
proslaviti = pro slav iti
proslavjati = pro slav jati
proslavjati = pro slav jati
proslavjati = pro slav jati
slaviti =  slav iti
slaviti =  slav iti
slaviti =  slav iti


In [520]:
morphemes.query("en.str.contains('suck')")
morphemes.loc[16929, 'base_verb'] = 'sȯsati'
manual_insert('sȯsati', 's[ȯoy]s', dry_run=0)

izsȯsati = iz sȯs ati
izsysati = iz sys ati
sȯsati =  sȯs ati
vsosati = v sos ati


In [479]:
# 'izmirati', 'zamirati', 'umreti', 'umirati'

set_base_verb(morphemes.query("base_verb.str.contains('mi?r[aeě]?ti')").sort_values(by='base_verb'), "mrěti")
manual_insert("mrěti", "mi?r[aeě]?", dry_run=0)

izmirati = iz mira ti
izmrěti = iz mrě ti
izumirati = izu mira ti
izumreti = izu mre ti
umirati = u mira ti
umreti = u mre ti
zamirati = za mira ti
zamreti = za mre ti


In [491]:
set_base_verb(morphemes.query("base_verb == 'oblěkti'"), 'vlěkti')
set_base_verb(morphemes.query("base_verb == 'oblåčiti'"), 'vlěkti')

manual_insert("sȯniti", 'sn', dry_run=0)
manual_insert("sȯvětovati", 'vět', dry_run=0)
morphemes.loc[10905, 'derived_nouns'] = 'věće'

sniti =  sn iti
usnųti = u sn ųti
zasnųti = za sn ųti
posȯvětovati = posȯ vět ovati


In [499]:
set_base_verb(morphemes.query("en.str.contains('dig') and (isv.str.contains('ryv') or base_verb.str.contains('rv'))"), "ryti")


In [501]:
morphemes.loc[8557, 'base_verb'] = 'rvati'

In [505]:
manual_insert("rvati", 'ry?v', dry_run=0)

manual_insert("ryti", 'ry', dry_run=0)

izrvati = iz rv ati
izryvati = iz ryv ati
narvati = na rv ati
naryvati = na ryv ati
obrvati = ob rv ati
obryvati = ob ryv ati
odrvati = od rv ati
odryvati = od ryv ati
prěrvati = prě rv ati
prěryvati = prě ryv ati
råzrvati = råz rv ati
råzryvati = råz ryv ati
rvati =  rv ati
rvati =  rv ati
rvati =  rv ati
urvati = u rv ati
uryvati = u ryv ati
odryti = od ry ti
odryvati = od ry vati
råzryti = raz ry ti
råzryvati = raz ry vati
ryti =  ry ti
vyryti = vy ry ti


In [510]:
set_base_verb(morphemes.query("base_verb.str.contains('^[o]?briti')"), "briti")
manual_insert("briti", 'br', dry_run=1)



briti =  br iti
briti =  br iti
obriti = o br iti
obriti = o br iti


In [500]:
#set_base_verb(
morphemes.query("isv.str.contains('ryv') and not base_verb.str.contains('igra') and not base_verb.str.contains('kryti')")#, "hråbriti")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4245,23866.0,izryvati,NaN,v.tr. ipf.,tear out,NaN,verb,?,?,rvati,izryvati,izr,?ryvati?,,,iz,ryv,ati,0
7419,33188.0,naryvati,NaN,v.tr. ipf.,"pick (flowers), gather (flowers)",NaN,verb,?,?,rvati,naryvati,nar,?ryvati?,,,na,ryv,ati,0
8557,28923.0,obryvati,NaN,v.tr. ipf.,"pick, pluck, tear off",NaN,verb,?,?,obriti,obryvati,obr,?ryvati?,,,ob,ry,vati,0
9006,29227.0,odryvati,NaN,v.tr. ipf.,dig up,NaN,verb,?,?,ryti,odryvati,odr,?ryvati?,,,od,ryv,ati,0
9007,34361.0,odryvati,NaN,v.tr. ipf.,tear away,NaN,verb,?,?,rvati,odryvati,odr,?ryvati?,,,od,ryv,ati,0
11506,1849.0,prěryvati,NaN,v.tr. ipf.,interrupt,NaN,verb,?,?,rvati,prěryvati,prěr,?ryvati?,,,prě,ryv,ati,0
12846,30446.0,råzryvati,NaN,v.tr. ipf.,"tear to pieces, tear apart, lacerate, disrupt",NaN,verb,råz’,ryvati,rvati,råzryvati,råzr,?ryvati?,,,råz,ryv,ati,0
12847,30447.0,råzryvati,NaN,v.tr. ipf.,"dig up, plough up, root up",NaN,verb,råz’,ryvati,ryti,råzryvati,råzr,?ryvati?,,,råz,ryv,ati,0
15843,23940.0,uryvati,NaN,v.tr. ipf.,"snatch, pluck, cull",NaN,verb,?,?,rvati,uryvati,ur,?ryvati?,,,u,ryv,ati,0


In [465]:
set_base_verb(morphemes.query("base_verb == 'tati'"), "stati")



In [431]:
morphemes.loc[9271, ["base_verb", '_stem', '_suffix']] = ['okupovati', 'okup', 'ovati']

In [449]:
set_base_verb(morphemes.query("isv.str.contains('hr.br')"), "hråbriti")
manual_insert("hråbriti", 'hr[aå]br', last_cons={"g", "ž"}, dry_run=0)

obezhråbriti = obez hråbr iti
obezhrabrjati = obez hrabr jati


In [452]:

morphemes.query("base_verb.str.contains('^niti')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
12732,30345.0,råzniti,NaN,v.refl. ipf.,differ,NaN,verb,råz’,niti sę,niti,råzniti sę,råzn,niti,|råznica,,,råzn,iti,0
12827,18618.0,råzråzniti,NaN,v.tr. pf.,distinguish,NaN,verb,råz’,råzniti,niti,råzråzniti,råzråzn,niti,,,raz,råzn,iti,0


In [446]:
set_base_verb(morphemes.query("isv.str.contains('valiti') and not isv.str.contains('hvaliti')"), 'valiti')
manual_insert("valiti", 'val', last_cons={"g", "ž"}, dry_run=0)

råzvaliti = raz val iti
råzvaliti = raz val iti
råzvaliti = raz val iti
råzvaliti = raz val iti
valiti =  val iti
zavaliti = za val iti


In [447]:
set_base_verb(morphemes.query("base_verb.str.contains('uvędati')"), 'vędnųti')
set_base_verb(morphemes.query("base_verb.str.contains('važ.ti')"), "važiti")
# morphemes.query("en.str.contains('weigh')")
manual_insert("važiti", 'vag', last_cons={"g", "ž"}, dry_run=0)

obvažati = ob vag ⒥ati
obvažiti = ob vag ⒥iti
podvažati = pod vag ⒥ati
podvažiti = pod vag ⒥iti
považati = po vag ⒥ati
považiti = po vag ⒥iti
prěvažati = prě vag ⒥ati
prěvažiti = prě vag ⒥iti
råzvažati = raz vag ⒥ati
råzvažiti = raz vag ⒥iti
uvažati = u vag ⒥ati
uvažati = u vag ⒥ati
uvažiti = u vag ⒥iti
važiti =  vag ⒥iti
zauvažati = zau vag ⒥ati
zauvažiti = zau vag ⒥iti


In [422]:
morphemes.loc[16132, 'base_verb'] = 'važiti'

In [395]:
g = morphemes.query("base_verb in ['vȯzȯjdti', 'najdti']")
set_base_verb(g, "i1ti")
manual_insert("i1ti", 'jd', dry_run=0)
set_base_verb(g, "idti")


iznajdti = izna jd ti
prěvȯzȯjdti = prěvȯzȯ jd ti


In [418]:
set_base_verb(morphemes.query("isv.str.contains('zdr[aå]v')"), "zdråvěti")
manual_insert("zdråvěti", "zdr[aå]v", dry_run=0)

nazdråviti = na zdråv iti
nazdravjati = na zdrav jati
ozdråvěti = o zdråv ěti
ozdråviti = o zdråv iti
ozdravjati = o zdrav jati
pozdråviti = po zdråv iti
pozdravjati = po zdrav jati
uzdråviti = u zdråv iti
uzdravjati = u zdrav jati
zdråvěti =  zdråv ěti


In [412]:
set_base_verb(morphemes.query("base_verb in ['zdråviti', 'uzdråviti']"), 'stojati')


nastojati = na stoj ati
nastojivati = na stoj ivati
obstajati = ob staj ati
obstajati = ob staj ati
pristojati = pri stoj ati
sȯstojati = sȯ stoj ati
stojati =  stoj ati
stojati =  stoj ati


In [413]:
manual_insert("vrěti", 'va?r', dry_run=0)
manual_insert("projektovati", 'projekt', dry_run=0)
manual_insert("programovati", 'program', dry_run=0)
manual_insert("drěmnųti", 'drěm', dry_run=0)
manual_insert("stojati", 'st[oa]j', dry_run=False)


variti =  var iti
vrěti =  vr ěti
zaprojektovati = za projekt ovati
prěprogramovati = prě program ovati
sdrěmnųti = s drěm nųti
nastojati = na stoj ati
nastojivati = na stoj ivati
obstajati = ob staj ati
obstajati = ob staj ati
pristojati = pri stoj ati
sȯstojati = sȯ stoj ati
stojati =  stoj ati
stojati =  stoj ati


In [376]:

set_base_verb(morphemes.query("isv.str.contains('směj')"), "smějati")
manual_insert("smějati", 'směj', dry_run=0)


izsmějati = iz směj ati
izsmějivati = iz směj ivati
råzsmějati = raz směj ati
smějati =  směj ati
zasmějati = za směj ati


In [370]:
set_base_verb(morphemes.query("isv.str.contains('jmati') or isv.str.contains('zęti$')"), 'imati')

#set_base_verb(morphemes.query("base_verb.str.contains('gȯlt')"), 'gȯltati')
manual_insert("imati", '[ij][mn]', dry_run=0)


imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
imati =  jm ati
izimati = iz jm ati
iznajmati = izna jm ati
najmati = na jm ati
najmati = na jm ati
obimati = ob jm ati
odimati = od jm ati
odimati = od jm ati
odimati = od jm ati
podimati = pod jm ati
pojmati = po jm ati
pozajmati = poza jm ati
prědprijmati = prědpri jm ati
prějmati = prě jm ati
prijmati = pri jm ati
prijmati = pri jm ati
projmati = pro jm ati
ujmati = u jm ati
vȯzimati = vȯz jm ati
vyjmati = vy jm ati
vzęti = vz in ⒩ti
zajmati = za jm ati
zajmati = za jm ati


In [355]:


set_base_verb(morphemes.query("base_verb.str.contains('gȯlt')"), 'gȯltati')
manual_insert("gȯltati", 'gȯlt', dry_run=0)

# morphemes.query("base_verb.str.contains('proje')")


gȯltati sȯlzy =  gȯlt ati
gȯltati =  gȯlt ati
gȯltnųti =  gȯlt nųti
pogȯltnųti sȯlzy = po gȯlt nųti
pogȯltnųti = po gȯlt nųti
progȯltati = pro gȯlt ati
progȯltnųti sȯlzy = pro gȯlt nųti
progȯltnųti = pro gȯlt nųti


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
3110,17418.0,gȯltati sȯlzy,NaN,v.ipf.,sob,NaN,verb,?,?,gȯltati,gȯltati sȯlzy,gȯltati sȯlzy,?gȯltati sȯlzy?,,,,gȯlt,ati,0
3111,5304.0,gȯltati,NaN,v.tr. ipf.,swallow,NaN,verb,?,?,gȯltati,gȯltati,gȯlt,gȯltati,|gȯltka,,,gȯlt,ati,0
3113,25650.0,gȯltnųti,NaN,v.tr. pf.,swallow,NaN,verb,?,?,gȯltati,gȯltnųti,gȯlt,gȯltnųti,|gȯltka,,,gȯlt,nųti,0
10533,18277.0,pogȯltnųti sȯlzy,NaN,v.pf.,sob,NaN,verb,?,?,gȯltati,pogȯltnųti sȯlzy,pogȯltnųti sȯlzy,?gȯltnųti sȯlzy?,,,po,gȯlt,nųti,0
10534,10800.0,pogȯltnųti,NaN,v.tr. pf.,swallow,NaN,verb,?,?,gȯltati,pogȯltnųti,pogȯlt,gȯltnųti,,,po,gȯlt,nųti,0
12012,21461.0,progȯltati,NaN,v.tr. ipf.,swallow,NaN,verb,?,?,gȯltati,progȯltati,progȯlt,gȯltati,,,pro,gȯlt,ati,0
12013,18413.0,progȯltnųti sȯlzy,NaN,v.pf.,sob,NaN,verb,?,?,gȯltati,progȯltnųti sȯlzy,progȯltnųti sȯlzy,?gȯltnųti sȯlzy?,,,pro,gȯlt,nųti,0
12014,18412.0,progȯltnųti,NaN,v.tr. pf.,swallow,NaN,verb,?,?,gȯltati,progȯltnųti,progȯlt,gȯltnųti,,,pro,gȯlt,nųti,0


In [356]:
morphemes.query("base_verb.str.contains('zami?r.ti')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
17301,33669.0,zamirati,NaN,v.intr. ipf.,"die away, fade away, come to a standstill",NaN,verb,?,?,zamirati,zamirati,zamir,?mirati?,,,za,mir,ati,0
17317,31527.0,zamreti,(zamre),v.intr. pf.,"die away, fade away, come to a standstill",NaN,verb,?,?,zamirati,zamreti,zamreti,?mreti?,,,za,mre,ti,0


In [352]:
def set_base_verb(g, base_verb):
    morphemes.loc[
        g.index,
        "base_verb"
    ] = base_verb


In [340]:
# {"sk", "šč"}
morphemes.loc[
    morphemes.query("base_verb == 'ščiti'").index,
    "base_verb"
] = "voščiti"
manual_insert("voščiti", 'vosk', last_cons={"k", "č"}, dry_run=0)


navoščiti = na vosk ⒥iti
ovoščiti = o vosk ⒥iti
voščiti =  vosk ⒥iti


In [268]:
list(ORPHANS)[3]

'råzsypati'

In [315]:

morphemes.loc[
    morphemes.query(" isv.str.contains('skub')").index,
    "base_verb"
] = "skubti"
manual_insert("skubti", 'skub', dry_run=0)


oskubati = o skub ati
oskubyvati = o skub yvati
skubnųti =  skub nųti
skubti =  skub ti


In [313]:
morphemes.query("_prefix.str.contains('s$') and base_verb in @ORPHANS")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
9562,29623.0,oskubati,NaN,v.tr. pf.,"pluck, deplume",NaN,verb,?,?,oskubati,oskubati,oskub,?kubati?,,,os,kub,ati,0
9563,34592.0,oskubyvati,NaN,v.tr. ipf.,"pluck, deplume",NaN,verb,?,?,oskubati,oskubyvati,oskub,?kubyvati?,,,os,kuby,vati,0
13764,3139.0,skraćati,NaN,v.tr. ipf.,"shorten, abbreviate",NaN,verb,?,?,skråtiti,skraćati,skrać,?kraćati?,,,s,krat,⒥ati,0
13767,647.0,skråtiti,NaN,v.tr. pf.,"shorten, abbreviate",NaN,verb,?,?,skråtiti,skråtiti,skråt,?kråtiti?,,,s,kråt,iti,0
14006,20715.0,smykati,NaN,v.tr. ipf.,close (ranks),NaN,verb,?,?,sȯmknųti,smykati,smyk,?mykati?,,,s,myk,ati,0


In [310]:
morphemes.query(" base_verb == 'sedl' or base_verb.str.contains('sedl')")


morphemes.loc[
    morphemes.query(" base_verb.str.contains('sedl')").index,
    "base_verb"
] = "sedlati"
manual_insert("sedlati", 'sedl', dry_run=0)


osedlati = o sedl ati
råzsedlati = raz sedl ati
råzsedlyvati = raz sedl yvati
sedlati =  sedl ati


In [308]:

morphemes.loc[
    morphemes.query(" base_verb.str.contains('syp')").index,
    "base_verb"
] = "sypati"
manual_insert("sypati", 'syp', dry_run=0)



izsypati = iz syp ati
nasypati = na syp ati
obsypati = ob syp ati
obsypyvati = ob syp yvati
råzsypati = raz syp ati
råzsypyvati = raz syp yvati
sypati =  syp ati


In [304]:
morphemes.loc[
    morphemes.query(" base_verb == 'vȯzsiliti'").index,
    "base_verb"
] = "siliti"
manual_insert("siliti", 'sil', dry_run=0)


usiliti = u sil iti
usiljati = u sil jati
vȯzsiliti = vȯz sil iti
vȯzsilovati = vȯz sil ovati


In [ ]:
    if word[-3:] == "ęti":
        word = word[:-3] + "in" + NASAL + "ti"
    if "trim" not in word and "klimat" not in word:
        word = word.replace("idti", "jdti").replace("imati", "jmati")
    if signature == {'s', 't'}:
#        if word[-3:] == "sti":
#            # rasti -> rastti
#            word = word[:-3] + "stti"


In [397]:
BASE = "slěditi"

def check_if_orphan(BASE):

    g = morphemes.query(" base_verb == @BASE")

    base_variants = set((g['_stem'] + g['_suffix']).unique())
    variants = set(base_variants)
    for (src, dst) in [
        ("t" + YI, "ć"), ("sk" + YI, 'šć'), ("st" + YI, 'šć'),
        ("d" + YI, "đ"), ("zd" + YI, "žđ"),
        ("k" + YI, "č"), ("c" + YI, "č"), ("sk" + YI, "šč"),
        ("g" + YI, "ž"), ("z" + YI, "ž"), 
        ("s" + YI, "š"), ("h" + YI, "š"), 
        (AO, "a"), (AO, "o"), (AO, "å"), 
        ("in" + NASAL, "ę"),
        ("tt", "t"),
        ("bv", "v"),
        ("jdti", "idti"), ("jmati", "imati")
        # žđ  "h" + YI ?
    ]:
        variants |= {v.replace(src, dst) for v in base_variants}
    return BASE not in variants


In [289]:
# morphemes.query(" isv.str.contains('slě')")

In [290]:
morphemes.loc[
    morphemes.query(" isv.str.contains('slěd')").index,
    "base_verb"
] = "slěditi"

manual_insert("slěditi", 'slěd', last_cons={"d", "đ"}, dry_run=0)

izslědovati = iz slěd ovati
naslěditi = na slěd iti
naslědovati = na slěd ovati
naslědovati = na slěd ovati
obslědovati = ob slěd ovati
poslědovati = po slěd ovati
prěslědovati = prě slěd ovati
prěslědovati = prě slěd ovati
råzslěditi = raz slěd iti
råzslědovati = raz slěd ovati
slěditi =  slěd iti
slěditi =  slěd iti
slědovati =  slěd ovati


In [285]:
manual_insert("naslěditi", 'slåd', last_cons={"d", "đ"}, dry_run=1)

naslěditi = naslěditi slåd 
naslědovati = naslědovati slåd 
naslědovati = naslědovati slåd 


In [276]:
manual_insert("slåditi", 'slåd', last_cons={"d", "đ"}, dry_run=0)

naslåditi = na slåd iti
naslåditi = na slåd iti
naslađati = na slåd ⒥ati
naslađati = na slåd ⒥ati
oslåditi = o slåd iti


In [266]:
morphemes.query(" base_verb == 'slåditi'")
manual_insert("končati", 'konc', last_cons={"c", "č"}, dry_run=0)

dokončiti = do konc ⒥iti
končati =  konc ⒥ati
končati =  konc ⒥ati
skončiti = s konc ⒥iti
skončiti = s konc ⒥iti
zakončiti = za konc ⒥iti


In [260]:

manual_insert("jęti", 'jin⒩', last_cons={"k", "č"}, dry_run=1)

izjęti = iz jin⒩ ti
iznajęti = izna jin⒩ ti
jęti =  jin⒩ ti
najęti = na jin⒩ ti
najęti = na jin⒩ ti
objęti = ob jin⒩ ti
odjęti = od jin⒩ ti
odjęti = od jin⒩ ti
odjęti = od jin⒩ ti
podjęti = pod jin⒩ ti
pojęti = po jin⒩ ti
pozajęti = poza jin⒩ ti
prědprijęti = prědpri jin⒩ ti
prějęti = prě jin⒩ ti
prijęti = pri jin⒩ ti
prijęti = pri jin⒩ ti
projęti = pro jin⒩ ti
sjęti = s jin⒩ ti
ujęti = u jin⒩ ti
vȯzjęti = vȯz jin⒩ ti
zajęti = za jin⒩ ti
zajęti = za jin⒩ ti


In [248]:
set((g['_stem'].replace(AO, "o") + g['_suffix']).unique())

{'birati',
 'bližati',
 'bližiti',
 'budovati',
 'běgati',
 'běgti',
 'davati',
 'dirati',
 'dreti',
 'dyhati',
 'dělati',
 'dȯhnųti',
 'edlati',
 'edlyvati',
 'gnesti',
 'gnųti',
 'grěšiti',
 'gybati',
 'hovati',
 'hranjati',
 'hråniti',
 'hudnųti',
 'hvatiti',
 'hꜵditi',
 'ikati',
 'iliti',
 'ilovati',
 'jedati',
 'jesti',
 'jin⒩ti',
 'kazati',
 'kazyvati',
 'klanjati',
 'klejiti',
 'kloniti',
 'kombinovati',
 'kompensovati',
 'komplikovati',
 'konfiskovati',
 'končiti',
 'koriti',
 'korjati',
 'krat⒥ati',
 'kriviti',
 'kryti',
 'kryvati',
 'kråtiti',
 'kubati',
 'kubyvati',
 'kuhati',
 'lad⒥ati',
 'lgati',
 'liti',
 'livati',
 'lomiti',
 'ložiti',
 'lučati',
 'lučiti',
 'låditi',
 'lěditi',
 'lědovati',
 'lěpiti',
 'mlåtiti',
 'mlěti',
 'moliti',
 'mrščiti',
 'mut⒥ati',
 'mykati',
 'mękčati',
 'mękčiti',
 'męčkati',
 'měriti',
 'měrjati',
 'měšati',
 'měšiti',
 'měšivati',
 'mųtiti',
 'mųt⒥ati',
 'mꜵgti',
 'nesti',
 'niziti',
 'niz⒥ati',
 'nositi',
 'padati',
 'pakovati',
 'paliti',

In [242]:
ORPHANS

{'',
 'brati',
 'hoditi',
 'jęti',
 'končati',
 'mogti',
 'naslěditi',
 'obsypati',
 'oskubati',
 'poriti',
 'prositi',
 'pręgti',
 'råsti',
 'råzsedlati',
 'råzslěditi',
 'råzsypati',
 'skråtiti',
 'slåditi',
 'stråniti',
 'sȯmknųti',
 'topiti',
 'tvoriti',
 'věděti',
 'vȯzsiliti',
 '†ložiti',
 '†niziti',
 '†pokojiti'}

In [241]:
morphemes.query("_stem == 'mꜵg'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4083,31423.0,izmagati,NaN,v.tr. ipf.,extort,NaN,verb,?,?,mogti,izmagati,izmag,?magati?,,,iz,mꜵg,ati,0
4106,33356.0,izmogti,NaN,v.tr. pf.,extort,NaN,verb,?,?,mogti,izmogti,izmogti,mogti,,,iz,mꜵg,ti,0
4127,26207.0,iznemogti,NaN,v.intr. pf.,collapse,NaN,verb,?,?,mogti,iznemogti,iznemogti,mogti,,,izne,mꜵg,ti,0
6779,910.0,mogti,NaN,v.aux. ipf.,"can, may, be able",NaN,verb,?,?,mogti,mogti,mogti,mogti,,,,mꜵg,ti,0
10709,2205.0,pomagati,(+3),v.intr. ipf.,"help, assist, aid",NaN,verb,?,?,mogti,pomagati,pomag,?magati?,,,po,mꜵg,ati,0
10733,2229.0,pomogti,(+3),v.intr. pf.,"help, assist, aid",NaN,verb,?,?,mogti,pomogti,pomogti,mogti,,,po,mꜵg,ti,0
11422,20627.0,prěmagati,NaN,v.tr. ipf.,"overcome, surmount",NaN,verb,?,?,mogti,prěmagati,prěmag,?magati?,,,prě,mꜵg,ati,0
11440,20633.0,prěmogti,NaN,v.tr. pf.,"overcome, surmount",NaN,verb,?,?,mogti,prěmogti,prěmogti,mogti,,,prě,mꜵg,ti,0
13972,30731.0,smogti,NaN,v.aux. pf.,"be able, manage",NaN,verb,?,?,mogti,smogti,smogti,mogti,,,s,mꜵg,ti,0
17917,-30731.0,smogti,NaN,v.tr. pf.,"manage to, be able to, cope, make it",NaN,verb,?,?,mogti,smogti,smogti,mogti,,,s,mꜵg,ti,0


In [234]:
morphemes.query("base_verb == 'kakati' or en.str.contains('jump')")#.base_verb.value_counts()

morphemes.loc[
    morphemes.query("isv.str.contains('(skak)|(skoč)')").index,
    "base_verb"
] = "skakati"
#.base_verb.value_counts()

manual_insert("skakati", 'sk[ao]k', last_cons={"k", "č"}, dry_run=0)

naskakati = na skak ati
naskakati = na skak ati
naskakati = na skak ati
naskočiti = na skok ⒥iti
naskočiti = na skok ⒥iti
naskočiti = na skok ⒥iti
odskakati = od skak ati
odskočiti = od skok ⒥iti
skakati =  skak ati
skočiti =  skok ⒥iti
vȯzskakati = vȯz skak ati
vȯzskočiti = vȯz skok ⒥iti
vskočiti = v skok ⒥iti


C:\Users\79165\AppData\Local\Temp\ipykernel_17612\2999229346.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  morphemes.query("isv.str.contains('(skak)|(skoč)')").index,


In [224]:
morphemes.query("base_verb.str.contains('uměti')")
manual_insert("uměti", 'um', dry_run=0)

doråzuměti = doraz um ěti
nedoråzuměti = nedoraz um ěti
obezuměti = obez um ěti
obezuměvati = obez um ěvati
poråzuměti = poraz um ěti
poråzuměvati = poraz um ěvati
råzuměti = raz um ěti
sråzuměti = sraz um ěti


In [206]:
morphemes.loc[10306, "_suffix"] = 'in⒩ti'

In [213]:
morphemes.loc[
    morphemes.query("base_verb.str.contains('vŕnųti')").index,
    '_is_irregular'] = 1



In [210]:
morphemes.query("_suffix.str.contains('(ę)|(in⒩)') or base_verb == 'počęti'")
morphemes.query("_suffix.str.contains('tati')")



C:\Users\79165\AppData\Local\Temp\ipykernel_17612\1274580575.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  morphemes.query("_suffix.str.contains('(ę)|(in⒩)') or base_verb == 'počęti'")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
11065,22659.0,povŕtati,NaN,v.tr. ipf.,turn (tr.),NaN,verb,?,?,vŕnųti,povŕtati,povŕt,?ŕtati?,,,pov,ŕ,tati,0


In [163]:
# morphemes.query("base_verb.str.contains('vl[åa]d') or base_verb.str.contains('vęz')")

morphemes.loc[
    morphemes.query("isv.str.contains('vl[åa]d')").index,
    "base_verb"
] = 'vladati'

In [171]:
morphemes.query("base_verb.str.contains('pokojiti')")


manual_insert("†pokojiti", 'pok[ao]j', dry_run=0)

bezpokojiti = bez pokoj iti
bezpokojiti = bez pokoj iti
nepokojiti = ne pokoj iti
nepokojiti = ne pokoj iti
obezpokojiti = obez pokoj iti
uspokajati = us pokaj ati
uspokojiti = us pokoj iti
zanepokojiti = zane pokoj iti


In [168]:

manual_insert("vęzati", 'vęz', dry_run=0)

navęzati = na vęz ati
navęzati = na vęz ati
navęzyvati = na vęz yvati
navęzyvati = na vęz yvati
obvęzati = ob vęz ati
obvęzati = ob vęz ati
obvęzyvati = ob vęz yvati
obvęzyvati = ob vęz yvati
odvęzati = od vęz ati
odvęzyvati = od vęz yvati
prěsvęzati = prěs vęz ati
prěsvęzyvati = prěs vęz yvati
privęzati = pri vęz ati
privęzyvati = pri vęz yvati
råzvęzati = raz vęz ati
råzvęzyvati = raz vęz yvati
svęzati šnurom = s vęz ati
svęzati = s vęz ati
svęzyvati šnurom = s vęz yvati
svęzyvati = s vęz yvati
uvęznųti = u vęz nųti
vęzati šnurom =  vęz ati
vęzati =  vęz ati
vęznųti =  vęz nųti
zavęzati = za vęz ati
zavęzyvati = za vęz yvati


In [182]:

manual_insert("plesti", 'ple[st]', dry_run=0)

odplesti = od ples ti
odpletati = od plet ati
plesti =  ples ti
råzplesti = råz ples ti
råzpletati = råz plet ati
splesti = s ples ti
vplesti = v ples ti
vpletati = v plet ati


In [179]:

manual_insert("piti", 'pi', dry_run=0)

izpiti = iz pi ti
napiti = na pi ti
napiti = na pi ti
napivati = na pi vati
napivati = na pi vati
opiti = o pi ti
opiti = o pi ti
opivati = o pi vati
opivati = o pi vati
piti =  pi ti
popiti = po pi ti
popivati = po pi vati
propiti = pro pi ti
propivati = pro pi vati
upiti = u pi ti
upivati = u pi vati
vpiti = v pi ti
vpivati = v pi vati


In [166]:
manual_insert("vladati", 'vl[åa]d', dry_run=0)

ovladnųti = o vlad nųti
ovladnųti = o vlad nųti
ovladyvati = o vlad yvati
ovladyvati = o vlad yvati
prěvladnųti = prě vlad nųti
prěvladyvati = prě vlad yvati
sȯvladnųti = sȯ vlad nųti
sȯvladnųti = sȯ vlad nųti
sȯvladyvati = sȯ vlad yvati
sȯvladyvati = sȯ vlad yvati
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
vladati, vlåděti =  vlad ati,
zavladnųti = za vlad nųti
zavladyvati = za vlad yvati


In [196]:
morphemes.query("_prefix in @pWTF and _suffix in @sWTF")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular


In [195]:
morphemes.query("base_verb == 'služiti'")

manual_insert("služiti", 'slug', last_cons={"g", "ž"}, dry_run=0)

obslugovati = ob slug ovati
obslužiti = ob slug ⒥iti
odslužiti = od slug ⒥iti
posluživati = po slug ⒥ivati
služiti =  slug ⒥iti


In [193]:


manual_insert("vlěkti", 'vl[ěå]k', last_cons={"k", "č"}, dry_run=0)

izvlěkati = iz vlěk ati
izvlěkti = iz vlěk ti
navlěkati = na vlěk ati
navlěkti = na vlěk ti
odvlåčivati = od vlåk ⒥ivati
odvlåčivati = od vlåk ⒥ivati
odvlěkati = od vlěk ati
odvlěkati = od vlěk ati
odvlěkti = od vlěk ti
odvlěkti = od vlěk ti
povlěkati za sobojų = po vlěk ati
povlěkti za sobojų = po vlěk ti
privlåčivati = pri vlåk ⒥ivati
privlěkati = pri vlěk ati
privlěkti = pri vlěk ti
vlåčiti =  vlåk ⒥iti
vlåčiti =  vlåk ⒥iti
vlěkti =  vlěk ti


In [ ]:
vladati

In [184]:
morphemes.query("(_suffix in @sWTF or _suffix == '')  and _is_irregular == 0 and not isv.str.contains('[/,]')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4336,763.0,izvaljnjati,NaN,v.tr. ipf.,"fire (terminate employment), sack, discharge, ...",NaN,verb,?,?,voliti,izvaljnjati,izvaljn,?aljnjati?,,,iz,vꜵl,jnjati,0
4359,22512.0,izvlastniti,NaN,v.tr. pf.,expropriate,NaN,verb,?,?,vlastniti,izvlastniti,izvlastn,vlastniti,,,iz,vlast,niti,0
4360,22511.0,izvlastnjati,NaN,v.tr. ipf.,expropriate,NaN,verb,?,?,vlastniti,izvlastnjati,izvlastn,?lastnjati?,,,iz,vlast,njati,0
4377,2956.0,izvoljniti,NaN,v.tr. pf.,"fire (terminate employment), sack, discharge, ...",NaN,verb,?,?,voliti,izvoljniti,izvoljn,?ljniti?,,,iz,vꜵl,jniti,0
9102,22538.0,odvlåčivati,NaN,v.tr. ipf.,drag away,NaN,verb,?,?,vlěkti,odvlåčivati,odvlåč,?låčivati?,,,odv,låk,⒥ivati,0
9103,23884.0,odvlåčivati,NaN,v.tr. ipf.,distract,NaN,verb,?,?,vlěkti,odvlåčivati,odvlåč,?låčivati?,,,odv,låk,⒥ivati,0
10897,17010.0,posluživati,(+5),v.refl. ipf.,"use, make use of",NaN,verb,?,?,služiti,posluživati sę,posluž,?luživati?,,,pos,lug,⒥ivati,0
11065,22659.0,povŕtati,NaN,v.tr. ipf.,turn (tr.),NaN,verb,?,?,vŕnųti,povŕtati,povŕt,?ŕtati?,,,pov,ŕ,tati,0
11910,23175.0,privlåčivati,NaN,v.tr. ipf.,attract,NaN,verb,?,?,vlěkti,privlåčivati,privlåč,?låčivati?,,,priv,låk,⒥ivati,0
11911,22501.0,privlastniti sobě,NaN,v.tr. pf.,"appropriate, arrogate",NaN,verb,?,?,vlastniti,privlastniti sobě,privlastniti sobě,?lastniti sobě?,,,pri,vlast,niti,0


In [135]:
g = morphemes.query("base_verb.str.contains('sěj')")
manual_insert('bojevati', "boj", dry_run=0)

bojevati =  boj evati


In [157]:
g = morphemes.query("base_verb.str.contains('sěj')")
morphemes.loc[g.index, 'base_verb'] = 'sějati'
manual_insert('sějati', "sěj", dry_run=0)

obsějati = ob sěj ati
obsějivati = ob sěj ivati
posějati = po sěj ati
råzsějati = raz sěj ati
råzsějivati = raz sěj ivati
sějati =  sěj ati
zasějati = za sěj ati


In [137]:
g = morphemes.query("isv.str.contains('maljevati')")
manual_insert_g(g, "malj", dry_run=0)

maljevati =  malj evati


In [107]:

manual_insert('sȯsati', "s[ȯy]s", dry_run=0)

izsȯsati = iz sȯs ati
izsysati = iz sys ati
sȯsati =  sȯs ati


In [154]:

manual_insert('izsmějati', "směj", dry_run=0)

izsmějati = iz směj ati
izsmějivati = iz směj ivati


In [129]:
#uvěkověčiti
morphemes.query("base_verb.str.contains('sųsrědotočati')")

manual_insert('sųsrědotočati', "srědotok", last_cons={'č', 'k'}, dry_run=0)


sųsrědotočati = sų srědotok ⒥ati
sųsrědotočati = sų srědotok ⒥ati
sųsrědotočiti = sų srědotok ⒥iti
sųsrědotočiti = sų srědotok ⒥iti


In [114]:
#uvěkověčiti
morphemes.query("isv.str.contains('točn')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
15940,35669.0,utočniti,NaN,v.tr. pf.,"specify, pinpoint, clarify, straighten out",NaN,verb,?,?,utočniti,utočniti,utočn,?točniti?,,,,utočn,iti,0
15941,35670.0,utočnjati,NaN,v.tr. ipf.,"specify, pinpoint, clarify, straighten out",NaN,verb,?,?,utočniti,utočnjati,utočn,?točnjati?,,,,utočn,jati,0


In [116]:

manual_insert('utočniti', "točn", dry_run=0)
manual_insert('odčajati', "čaj", dry_run=0)


utočniti = u točn iti
utočnjati = u točn jati
odčajati = od čaj ati
odčajati = od čaj ati
odčajivati = od čaj ivati
odčajivati = od čaj ivati


In [84]:

morphemes.query("en.str.contains('etern')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular


In [92]:
manual_insert_g(
    morphemes.query("isv.str.contains('věk')"), 
    "věkověk", 
    last_cons={"k", "č"},
dry_run=False)

uvěkověčiti = u věkověk ⒥iti


In [ ]:
morphemes.query("base_verb == 'praviti'")

In [67]:
morphemes.loc[
    morphemes.query("en.str.contains('testif')").index,
    "base_verb"
] = "svědčiti"

In [89]:

manual_insert('svědčiti', "svědk",     last_cons={"k", "č"}, dry_run=False)

posvědčiti = po svědk ⒥iti
svědčiti =  svědk ⒥iti


In [66]:
morphemes.query("isv.str.contains('vědk')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular


In [63]:
g = morphemes.query("isv.str.contains('věd[oa]m')")

manual_insert_g(g, "věd[oa]m", dry_run=False)

osvědamjati = os vědam jati
osvědomiti = os vědom iti
uvědamjati = u vědam jati
uvědomiti = u vědom iti


In [56]:
manual_insert("sråmiti", "sråm", dry_run=False)

osråmiti = o sråm iti
osråmiti = o sråm iti
zasråmiti = za sråm iti
zasramjati = za sråm jati


In [ ]:
morphemes.query("_stem == 'ꜵtk' or base_verb == 'sråmiti'")


In [34]:
morphemes.query("_stem.str.contains('t[y]?k')").base_verb.unique()

array(['tknųti', 'tykati', 'tkati'], dtype=object)

In [79]:
manual_insert("praviti", "prav", dry_run=False)


izpraviti = iz prav iti
izpravjati = iz prav jati
napraviti = na prav iti
napraviti = na prav iti
napraviti = na prav iti
napraviti = na prav iti
napravjati = na prav jati
napravjati = na prav jati
napravjati = na prav jati
napravjati = na prav jati
odpraviti = od prav iti
odpraviti = od prav iti
odpraviti = od prav iti
odpravjati = od prav jati
odpravjati = od prav jati
odpravjati = od prav jati
opraviti = o prav iti
opraviti = o prav iti
opravjati = o prav jati
opravjati = o prav jati
popraviti = po prav iti
popravjati = po prav jati
praviti =  prav iti
prěpraviti = prě prav iti
prěpravjati = prě prav jati
pripraviti = pri prav iti
pripraviti = pri prav iti
pripravjati = pri prav jati
pripravjati = pri prav jati
upraviti = u prav iti
upravjati = u prav jati
upravjati = u prav jati
zapraviti = za prav iti
zapravjati = za prav jati


In [36]:


morphemes.loc[
    morphemes.query("base_verb == 'tknųti' or base_verb == 'tykati'").index,
    'base_verb'    
] = 'tykati'


In [41]:
import re

re.split('t[y]?k', "spotykati")

['spo', 'ati']

In [44]:
manual_insert("tykati", "t[y]?k", dry_run=True)

IndexError: list index out of range

In [6]:
sWTF

Index(['ijati', 'njati', 'inųti', 'jevati', 'amjati', 'omiti', 'ajivati',
       'ipravjati', 'ipraviti', 'jivati', 'vovati', 'čiti', 'ropivati',
       'ropiti', 'opivati', 'ěpravjati', 'ěpraviti', 'kovati', 'opiti', 'ęti',
       'iveti', 'ověti', 'ověkյiti'],
      dtype='object')

In [ ]:
verb_prefixes._suffix = verb_prefixes._suffix.str.replace(YI, "⒥").str.replace(NASAL, "⒩")

verb_prefixes[['id', 'isv_orig', 'addition', 'partOfSpeech', 'en', 'genesis',
       'base_verb', 'derived_nouns', 'base_noun', '_prefix', '_stem',
       '_suffix', '_is_irregular']].to_csv('verb_morphemes.csv')
verb_prefixes.to_pickle("verb_prefixes_last.pkl")

In [706]:
morphemes[morphemes.isv != morphemes._prefix + morphemes._stem + morphemes._suffix].query(
    "not _stem.str.contains('ꜵ') and not _suffix.str.contains('[⒥⒩]') and not isv.str.contains('[ \/]')"
)

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2179,7211.0,dorastati,NaN,v.intr. ipf.,"grow up, become an adult",NaN,verb,?,?,råsti,dorastati,dorast,?rastati?,,,do,råst,ati,0
2180,16416.0,doråsti,(doråste),v.intr. pf.,"grow up, become an adult",NaN,verb,?,?,råsti,doråsti,doråsti,råsti,,,do,råst,ti,0
3640,2104.0,idti,(ide; šel),v.intr. ipf.,go,NaN,verb,?,?,idti,idti,idti,idti,,,,jd,ti,0
3685,3325.0,imati,NaN,v.ipf.,deal with,NaN,verb,?,?,imati,iměti dělo s,iměti dělo s,?iměti dělo s?,,,,jm,ati,0
3686,36890.0,imati,NaN,v.tr. ipf.,"mean, have in mind",NaN,verb,?,?,imati,iměti na myslji,iměti na myslji,?iměti na myslji?,,,,jm,ati,0
3687,36928.0,imati,NaN,v.tr. ipf.,"mean, have in mind",NaN,verb,?,?,imati,iměti na umu,iměti na umu,?iměti na umu?,,,,jm,ati,0
3688,17485.0,imati,NaN,v.ipf.,hope,NaN,verb,?,?,imati,iměti nadějų,iměti nadějų,?iměti nadějų?,,,,jm,ati,0
3690,876.0,imati,NaN,v.ipf.,be right,NaN,verb,?,?,imati,iměti pravdų,iměti pravdų,?iměti pravdų?,,,,jm,ati,0
3691,36402.0,imati,NaN,v.ipf.,make sense,NaN,verb,?,?,imati,iměti smysl,iměti smysl,?iměti smysl?,,,,jm,ati,0
3692,23339.0,imati,NaN,v.intr. ipf.,matter,NaN,verb,?,?,imati,iměti važnosť,iměti važnosť,?iměti važnosť?,,,,jm,ati,0


In [709]:
!ls -lh | grep wik

-rw-r--r-- 1 79165 197609  230K Mar 26 00:41 data gathering-wiktionary_new.ipynb
-rw-r--r-- 1 79165 197609  7.9M Nov 11 20:36 isv_wiki.pkl
-rw-r--r-- 1 79165 197609   33K Nov 11 23:16 isv_wiki_crawling.ipynb
-rw-r--r-- 1 79165 197609  412K May  6 00:00 morphemes_plus_wiktionary.ipynb
drwxr-xr-x 1 79165 197609     0 Nov 11 23:16 wiki
-rw-r--r-- 1 79165 197609  1.5M Nov  8 00:48 wiktionary.json
-rw-r--r-- 1 79165 197609  342K Nov  8 00:49 wiktionary.zip
-rw-r--r-- 1 79165 197609   16M Nov 30 04:20 wiktionary_extended.json
-rw-r--r-- 1 79165 197609  7.5M Mar 25 04:16 wiktionary_extended2.json
-rw-r--r-- 1 79165 197609   12M Mar 26 00:31 wiktionary_extended_new.json


In [725]:
from isv_nlp_utils.flavorization.replacer import VOWELS
from isv_nlp_utils.flavorization.parsing import parse_multireplacer_rules
from isv_nlp_utils.flavorization.tokenizer import compute_annotated_tokens, pretty_stringify
from isv_nlp_utils.flavorization.replacer import process_multireplacing, morphological_flavorise
from isv_nlp_utils.flavorization.selector import produce_string, filter_good_spellings, filter_lingua, init_detector, init_hunspell
from isv_nlp_utils.flavorization.tokenizer import tokens_to_exhaustive_string_list

In [728]:
from razdel import tokenize
from isv_nlp_utils.flavorization.tokenizer import ParseVariant, AnnotatedToken

In [726]:
rules_struct, declared_constants = parse_multireplacer_rules(
        r"C:\dev\razumlivost\src\flavorizers\slow\protoslavic.ts"
)

def f(word, rules_struct, declared_constants, pos_tag):
    cap = False
    space_after = ""
    
    slovnik_pos = ""
    isv_lemma = None
    variants = [ParseVariant(
                    [word],
                    pos_tag, slovnik_pos, isv_lemma,
                    None, "", 
                    False
                )]

    ann_token = AnnotatedToken(
        variants,
        cap, space_after,
    )
    
    tokens_base = [ann_token]
    # tokens = morphological_flavorise(tokens_base, morph, flavor_rules[LANG])
    tokens = process_multireplacing(tokens_base, rules_struct, declared_constants)
    return tokens

In [729]:
word = 'bolь'


pretty_stringify(
    f(word, rules_struct, declared_constants, {""})
)

'bolj'

In [ ]:
all_reconstructions = set()
reconstructed_articles = {}

for k, v in words_data.items():
    if v['Related_Slavic']:
        all_reconstructions |= set([x[1][1:] for x in v['Related_Slavic']])
    all_reconstructions.add(v['*'])
    reconstructed_articles[v['*']] = v

len(all_reconstructions), len(reconstructed_articles)

In [834]:
pra_verbs = {}
pra_forms = {}

all_reconstructions = set()

for k, v in words_data.items():
    if v['Related_Slavic']:
        all_reconstructions |= set([x[1][1:] for x in v['Related_Slavic']])
    all_reconstructions.add(v['*'])

for word in all_reconstructions:
    if word.endswith('ti') or word.endswith('ťi'):

        evolved = pretty_stringify(
            f(word, rules_struct, declared_constants, {""})
        )
        if "|" in evolved:
            variants = evolved[1:-1].split("|")
            variants = [vr for vr in variants if vr in morphemes.isv.unique()]
            if len(variants) == 1:
                pra_verbs[word] = variants[0]
                pra_forms[variants[0]] = word
            else:
                print(word, evolved, variants)
        else:
            pra_verbs[word] = evolved
            pra_forms[evolved] = word

o(b)klasti [obklasti|oklasti] []
postiťi [postikti|postigti] []
pľuti [pljuti|pjuti] []
-ťi [-gti|-kti] []
bľukati [bljukati|bjukati] []
orzkošiti [razkošiti|råzkošiti|rozkošiti] []
sęťi [sękti|sęgti] []
orzьniti [roźniti|raźniti|råźniti] []
orzduxati [rozduhati|razduhati|råzduhati] []
povelťi [povlěgti|povlěkti] []
(s)tъrčati [strčati|trčati] []
orsěsti [råsěsti|rasěsti|rosěsti] []
orzžaliti [razžaliti|råzžaliti|rozžaliti] []
pręťi [prękti|pręgti] []
poteťi [potegti|potekti] []
ortiti [ratiti|råtiti|rotiti] []
žegťi [žeggti|žegkti] []
orsuti [råsuti|rosuti|rasuti] []
o(b)teťi [otekti|otegti|obtekti|obtegti] []
skobľati [skobljati|skobjati] []
dostiťi [dostigti|dostikti] []
napeťi [napegti|napekti] []
bel(e)niti [beleniti|blěniti] []
jьzreťi [izregti|izrekti] []
sъpeťi [spekti|spegti] []
ortati [råtati|rotati|ratati] []
orzuzdati [razuzdati|råzuzdati|rozuzdati] []
tęťi [tęgti|tękti] []
orzvětriti [rozvětriti|razvětriti|råzvětriti] []
stiťi [stikti|stigti] []
o(b)kladati [okladati|obkla

In [835]:
len(pra_forms)

2845

In [739]:
# pra_verbs

In [836]:
"" in pra_forms

False

In [907]:
morphemes['reconstructed'] = ""

In [908]:
from collections import Counter

cnt = Counter()

unk = []
dfs = []
for nest in morphemes.base_verb.unique():
    g = morphemes.query('base_verb == @nest')
    if nest.strip("†") in pra_forms and nest not in morphemes.isv.unique():
        print("†?", nest, pra_forms[nest.strip("†")])
        morphemes.loc[g.index, 'reconstructed'] = "?" + pra_forms[nest.strip("†")]
        continue

    g2 = g.isv.apply(lambda x: pra_forms.get(x, float("nan")))
    morphemes.loc[g.index, 'reconstructed'] = g2.values
    cnt[len(g2.dropna().unique())] += 1
    if len(g2.dropna().unique()) == 0:
        unk.append(nest)
        continue
    if len(g2.dropna()) != len(g2):
        if len(g2.dropna().unique()) > 1:
            dfs.append(g)
        selected = min(g2.dropna().unique(), key=len)
        g3 = morphemes.loc[g.index].query("isv not in @pra_forms")
        morphemes.loc[g3.index, 'reconstructed'] = "?" + selected


cnt

†? sloviti sloviti
†? †kladati kladati
†? †ložiti ložiti
†? voliti voliti
†? †gynųti gynǫti
†? †městiti městiti
†? mrěti merti
†? †niknųti niknǫti
†? obraziti obraziti
†? †plåšiti polšiti
†? pęti pęti
†? poriti poriti
†? světiti světiti
†? viniti viniti
†? †vysiti vysiti
†? †spěti spěti
†? †strěti sterti
†? †trčati tъrčati
†? †pitati pitati


Counter({0: 776, 1: 400, 3: 17, 2: 78, 6: 4, 27: 1, 4: 8, 5: 2, 8: 1, 7: 1})

In [854]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
17905,19473.0,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,,,,žuž,ati,0,žužati


In [843]:
cnt

Counter({0: 779, 1: 401, 3: 17, 2: 78, 6: 4, 27: 1, 4: 8, 5: 2, 8: 1, 7: 1})

'None'

In [794]:
g2

17905    None
Name: isv, dtype: object

In [909]:
t1 = morphemes.dropna(subset=['reconstructed']).query('reconstructed.str.contains("\?")')

t1

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
887,4889.0,blågodariti,NaN,v.tr. ipf.,thank,NaN,verb,?,?,dariti,blågodariti,blågodar,blågodariti,,,blågo,dar,iti,0,?dariti
899,5574.0,blågoslavjati,NaN,v.tr. ipf.,"bless, beatify",NaN,verb,?,?,sloviti,blågoslavjati,blågoslav,blågoslavjati,,,blågo,slav,jati,0,?sloviti
900,5573.0,blågosloviti,NaN,v.tr. pf.,"bless, beatify",NaN,verb,?,?,sloviti,blågosloviti,blågosl,blågosloviti,,,blågo,slov,iti,0,?sloviti
912,6050.0,blågoželati,(+3),v.intr. ipf.,congratulate,NaN,verb,?,?,želati,blågoželati,blågožel,blågoželati,,,blågo,žel,ati,0,?želati
988,36422.0,bodnųti,(bode),v.refl. pf.,lock horns,NaN,verb,?,?,bodati,bodnųti sę,bod,bodnųti,|bodec,,,bod,nųti,0,?bodati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17581,32633.0,zažigati,NaN,v.tr. ipf.,"ignite, light, kindle, inflame",NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,za,žig,ati,0,?žeťi
17582,32636.0,zažigati,NaN,v.tr. ipf.,"switch on (the lights), turn on (the light)",NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,za,žig,ati,0,?žeťi
17583,32638.0,zažigati,NaN,v.tr. ipf.,set fire to,NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,za,žig,ati,0,?žeťi
17624,23718.0,zěvnųti,NaN,v.intr. pf.,yawn,NaN,verb,?,?,zěvati,zěvnųti,zěv,zěvnųti,,,,zěv,nųti,-1,?zěvati


In [910]:
morphemes.loc[
    t1[(t1._prefix + t1.reconstructed.str.replace("?", "").apply(pra_verbs.get) == t1.isv)].index,
    "reconstructed"
] = "(" + t1._prefix + ")+" + t1.reconstructed.str.replace("?", "")


C:\Users\79165\AppData\Local\Temp\ipykernel_17612\2634797297.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ] = "(" + t1._prefix + ")+" + t1.reconstructed.str.replace("?", "")
C:\Users\79165\AppData\Local\Temp\ipykernel_17612\2634797297.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  t1[(t1._prefix + t1.reconstructed.str.replace("?", "").apply(pra_verbs.get) == t1.isv)].index,
C:\Users\79165\AppData\Local\Temp\ipykernel_17612\2634797297.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  morphemes.loc[


In [911]:
morphemes.query("reconstructed != reconstructed and genesis != genesis").tail(55)

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
17464,23842.0,zasramjati,NaN,v.tr. ipf.,embarrass,NaN,verb,?,?,sråmiti,zasramjati,zasram,?ramjati?,,,za,sråm,jati,0,NaN
17466,1932.0,zasrati,NaN,v.intr. pf.,shit,NaN,verb,?,?,srati,zasrati,zasr,srati,,,za,sr,ati,0,NaN
17468,4112.0,zastariti,NaN,v.intr. pf.,age,NaN,verb,?,?,zastariti,zastariti,zastar,?tariti?,,,,zastar,iti,0,NaN
17469,4111.0,zastarjati,NaN,v.intr. ipf.,age,NaN,verb,?,?,zastariti,zastarjati,zastar,?tarjati?,,,,zastar,jati,0,NaN
17478,21834.0,zastavjati,NaN,v.tr. ipf.,"block, obstruct, cram",NaN,verb,?,?,stavjati,zastavjati,zastav,stavjati,,,za,st,avjati,0,NaN
17479,21840.0,zastavjati,NaN,v.tr. ipf.,"pawn, hock",NaN,verb,?,?,stavjati,zastavjati,zastav,stavjati,,,za,st,avjati,0,NaN
17483,19418.0,zastrašati,NaN,v.tr. ipf.,intimidate,NaN,verb,?,?,zastrašati,zastrašati,zastraš,?trašati?,,,,zastraš,ati,0,NaN
17484,19419.0,zastrašiti,NaN,v.tr. pf.,intimidate,NaN,verb,?,?,zastrašati,zastrašiti,zastraš,strašiti,,,,zastraš,iti,0,NaN
17485,3268.0,zastrěliti,NaN,v.tr. pf.,"fire, shoot",NaN,verb,?,?,zastrěliti,zastrěliti,zastrěl,strěliti,,,,zastrěl,iti,0,NaN
17486,5113.0,zastrěljati,NaN,v.tr. ipf.,"fire, shoot",NaN,verb,?,?,zastrěliti,zastrěljati,zastrěl,strěljati,,,,zastrěl,jati,0,NaN


In [859]:
t1[(t1._prefix + t1.reconstructed.str.replace("?", "").apply(pra_verbs.get) != t1.isv)].query('not _suffix.str.contains("nųti")')

C:\Users\79165\AppData\Local\Temp\ipykernel_17612\2727962516.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  t1[(t1._prefix + t1.reconstructed.str.replace("?", "").apply(pra_verbs.get) != t1.isv)].query('not _suffix.str.contains("nųti")')


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
1185,22979.0,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,,,,,0,?uměti
1189,34791.0,buditi sę,NaN,v.tr. ipf.,wake up,NaN,verb,?,?,buditi,buditi sę,bud,?buditi sę?,|budka,,,bud,iti,0,?buditi
1246,35085.0,byti dȯlžen,NaN,v.aux. ipf.,"must, have to, ought to, should, be obliged to...",NaN,verb,?,?,byti,byti dȯlžen,byti dȯlžen,?byti dȯlžen?,,,,by,ti,0,?byti
1248,10343.0,byti ostråžny,NaN,v.ipf.,"look out, watch out",NaN,verb,?,?,byti,byti ostråžny,byti ostråžny,?byti ostråžny?,,,,by,ti,0,?byti
1249,17182.0,byti podobny,NaN,v.ipf.,"look alike, resemble",NaN,verb,?,?,byti,byti podobny,byti podobny,?byti podobny?,,,,by,ti,0,?byti
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17551,36089.0,zaviděti,(zavidi),v.intr. ipf.,envy,NaN,verb,?,?,viděti,zaviděti,zavid,viděti,,,za,vid,ěti,0,?nenaviděti
17553,20321.0,zavisěti od,(zavisi),v.intr. ipf.,depend on,NaN,verb,?,?,visěti,zavisěti od,zavisěti od,?isěti od?,,,za,vis,ěti,0,?visěti
17581,32633.0,zažigati,NaN,v.tr. ipf.,"ignite, light, kindle, inflame",NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,za,žig,ati,0,?žeťi
17582,32636.0,zažigati,NaN,v.tr. ipf.,"switch on (the lights), turn on (the light)",NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,za,žig,ati,0,?žeťi


In [860]:
morphemes[morphemes.base_verb == ''].query('isv in @pra_forms')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
7246,6087.0,nakladati,NaN,v.tr. ipf.,impose,NaN,verb,?,?,,nakladati,naklad,?kladati?,,naklad,,naklad,ati,0,nakladati
7247,28028.0,nakladati,NaN,v.tr. ipf.,load up,NaN,verb,?,?,,nakladati,naklad,?kladati?,,naklad,,naklad,ati,0,nakladati
8290,9775.0,obědati,NaN,v.intr. ipf./pf.,"dine, eat lunch",NaN,verb,?,?,,obědati,oběd,?ědati?,,oběd,,oběd,ati,0,obědati
8510,22681.0,obråtiti,(+3),v.refl. pf.,"address, appeal to",NaN,verb,?,?,,obråtiti sę k,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti
8511,22675.0,obråtiti,NaN,v.refl. pf.,"turn (intr.), turn around",NaN,verb,?,?,,obråtiti sę,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti
8512,22677.0,obråtiti,NaN,v.refl. pf.,turn (into sth.),NaN,verb,?,?,,obråtiti sę,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti
8513,22679.0,obråtiti,NaN,v.refl. pf.,convert (to a religion),NaN,verb,?,?,,obråtiti sę,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti
8514,3209.0,obråtiti,NaN,v.tr. pf.,turn around,NaN,verb,?,?,,obråtiti,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti
8515,22666.0,obråtiti,NaN,v.tr. pf.,"rotate, revolve (tr.)",NaN,verb,?,?,,obråtiti,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti
8516,22668.0,obråtiti,NaN,v.tr. pf.,convert (sth. into sth.),NaN,verb,?,?,,obråtiti,obråt,?råtiti?,,obråt,,obråt,iti,0,ob(v)ortiti


In [861]:
morphemes.reconstructed.value_counts()

?uměti          80
?stǫpati        27
?voliti         19
?obelťi         18
?praviti        17
                ..
(od)+aviti       1
(od)+jьgrati     1
(od)+xoditi      1
(od)+gybati      1
žužati           1
Name: reconstructed, Length: 1936, dtype: int64

In [912]:
len(morphemes), len(morphemes.query('reconstructed == reconstructed or genesis == genesis'))

(4985, 3463)

In [874]:
morphemes.query('(reconstructed != reconstructed) and (genesis != genesis)').head(333).tail(77)

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
5859,20420.0,lęgati,NaN,v.tr. ipf.,hatch,NaN,verb,?,?,lęgati,lęgati,lęg,lęgati,,,,lęg,ati,0,NaN
5894,27115.0,lěniti,NaN,v.refl. ipf.,be lazy,NaN,verb,?,?,lěniti,lěniti sę,lěn,lěniti,,,,lěn,iti,0,NaN
5903,27127.0,lepetati,NaN,v.intr. ipf.,"babble, prattle",NaN,verb,?,?,lepetati,lepetati,lepet,lepetati,,,,lepet,ati,0,NaN
5920,2655.0,leskati,NaN,v.intr. ipf.,shine,NaN,verb,?,?,leskati,leskati,lesk,leskati,,!lěska,,lesk,ati,0,NaN
5921,2697.0,lesknųti,NaN,v.intr. pf.,shine,NaN,verb,?,?,leskati,lesknųti,lesk,lesknųti,,,,lesk,nųti,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7449,28189.0,naslåditi,NaN,v.refl. pf.,"enjoy, gloat, take pleasure in",NaN,verb,?,?,slåditi,naslåditi sę,naslåd,slåditi,,,na,slåd,iti,0,NaN
7450,28187.0,naslåditi,NaN,v.tr. pf.,give pleasure to,NaN,verb,?,?,slåditi,naslåditi,naslåd,slåditi,,,na,slåd,iti,0,NaN
7451,33275.0,naslađati,NaN,v.refl. ipf.,"enjoy, gloat, take pleasure in",NaN,verb,?,?,slåditi,naslađati sę,naslađ,?lađati?,,,na,slåd,⒥ati,0,NaN
7452,33278.0,naslađati,NaN,v.tr. ipf.,give pleasure to,NaN,verb,?,?,slåditi,naslađati,naslađ,?lađati?,,,na,slåd,⒥ati,0,NaN


In [873]:
morphemes.query("base_noun in @all_reconstructions")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
712,2880.0,besědovati,NaN,v.intr. ipf.,"converse, chat",NaN,verb,?,?,besědovati,besědovati,besěd,besědovati,,besěda,,besědo,vati,0,besědovati
1292,24918.0,cěniti,NaN,v.tr. ipf.,appreciate,NaN,verb,?,?,cěniti,cěniti,cěn,cěniti,,cěna,,cěn,iti,0,cěniti
3138,401.0,gorěti,(gori),v.intr. ipf.,"burn, be on fire",NaN,verb,?,?,gorěti,gorěti,gor,gorěti,|gorka|gornik,gora,,gꜵr,ěti,0,gorěti
3283,17443.0,gromaditi,NaN,v.tr. ipf.,accumulate,NaN,verb,?,?,gromaditi,gromaditi,gromad,gromaditi,,gromada,,gromad,iti,0,NaN
3293,2568.0,groziti,NaN,v.intr. ipf.,threaten,NaN,verb,?,?,groziti,groziti,groz,groziti,,groza,,grꜵz,iti,0,groziti
3306,3950.0,grupovati,NaN,v.tr. ipf.,group,I,verb,?,?,grupovati,grupovati,grup,grupovati,,grupa,,grupo,vati,0,NaN
4487,1592.0,jedati,NaN,v.intr. ipf.,eat,NaN,verb,?,?,jedati,jedati,jed,jedati,,jeda,,jed,ati,0,NaN
4804,2565.0,karati,NaN,v.tr. ipf.,punish,NaN,verb,?,?,karati,karati,kar,karati,,kara,,kar,ati,0,karati
4805,35408.0,karati,NaN,v.tr. ipf.,"scold, rebuke, reprimand, reproach",NaN,verb,?,?,karati,karati,kar,karati,,kara,,kar,ati,0,karati
4958,26555.0,klevetati,NaN,v.intr. ipf.,slander,NaN,verb,?,?,klevetati,klevetati,klevet,klevetati,|klevetnik,kleveta,,klevet,ati,0,klevetati


In [779]:
morphemes.head(1).values

array([[11.0, 'abdikovati', nan, 'v.intr. ipf.', 'abdicate', 'I', 'verb',
        '?', '?', 'abdikovati', 'abdikovati', 'abdik', 'abdikovati',
        '|abdikacija', '', '', 'abdiko', 'vati', 0, None]], dtype=object)

In [798]:
BV = morphemes.query('reconstructed == reconstructed').base_verb.unique()

In [799]:
len(BV)

386

In [801]:
morphemes.query('reconstructed != reconstructed and base_verb in @BV and not isv.str.contains(" ")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
1185,22979.0,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,,,,,0,NaN
1710,25144.0,debatovati,NaN,v.intr. ipf.,debate,I,verb,’,debatovati,,debatovati,debat,?batovati?,,debata,,debat,ovati,0,NaN
1716,25152.0,debjutovati,NaN,v.intr. ipf./pf.,make one’s debut,F,verb,’,debjutovati,,debjutovati,debjut,?bjutovati?,,debjut,,debjut,ovati,0,NaN
1735,15280.0,defisovati,NaN,v.tr. ipf.,hyphenate,I,verb,’,defisovati,,defisovati,defis,?fisovati?,,defis,,defis,ovati,0,NaN
1988,25273.0,dobrěti,NaN,v.intr. ipf.,become good,NaN,verb,?,?,,dobrěti,dobr,?brěti?,|dobrosť,dobry,,dobr,ěti,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17500,23255.0,zašiti,(zašije),v.tr. pf.,"sew, stitch",NaN,verb,?,?,šiti,zašiti,zaš,šiti,,,za,š,iti,-1,NaN
17521,5204.0,zatvarjati,NaN,v.tr. ipf.,"close, shut",NaN,verb,?,?,tvoriti,zatvarjati,zatvar,?tvarjati?,,,za,tvꜵr,jati,0,NaN
17524,5205.0,zatvoriti,NaN,v.tr. pf.,"close, shut",NaN,verb,?,?,tvoriti,zatvoriti,zatvor,tvoriti,|zatvorka,,za,tvꜵr,iti,0,NaN
17537,14193.0,zautrakati,NaN,v.intr. ipf.,"eat breakfast, have breakfrast",NaN,verb,?,?,,zautrakati,zautrak,?trakati?,,zautraka,,zautrak,ati,0,NaN


In [758]:
pd.concat(dfs).sort_values(by='base_verb')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
12589,21505.0,raziti,NaN,v.tr. ipf.,"hit, strike",NaN,verb,?,?,,raziti,raz,?iti?,,raz,,raz,iti,0,
13330,12075.0,saditi,NaN,v.tr. ipf.,"plant, seat",NaN,verb,?,?,,saditi,sad,?aditi?,,sad,,sad,iti,0,
14037,34003.0,snovati,NaN,v.tr. ipf.,warp,NaN,verb,?,?,,snovati,sn,?novati?,,sn,,sn,ovati,0,
14619,2097.0,sųditi,NaN,v.tr. ipf.,"judge, try (in court)",NaN,verb,?,?,,sųditi,sųd,?diti?,,sųd,,sųd,iti,0,
14620,22003.0,sųditi,NaN,v.tr. ipf.,"predestine, foreordain",NaN,verb,?,?,,sųditi,sųd,?diti?,,sųd,,sųd,iti,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14699,18925.0,sušiti,NaN,v.tr. ipf.,dry,NaN,verb,?,?,šiti,sušiti,suš,šiti,,,,suh,⒥iti,0,
17846,7191.0,žiti,(žive),v.intr. ipf.,"reside, dwell",NaN,verb,?,?,žiti,žiti,ž,žiti,,,,ži,ti,-1,
17845,28.0,žiti,(žive),v.intr. ipf.,live,NaN,verb,?,?,žiti,žiti,ž,žiti,,,,ži,ti,-1,
11645,11412.0,prěživati,NaN,v.tr. ipf.,live through,NaN,verb,?,?,žiti,prěživati,prěž,?živati?,,,prě,ži,vati,-1,


In [751]:
unk

['abdikovati',
 'abonovati',
 'absorbovati',
 'abstrahovati',
 'adaptovati',
 'administrovati',
 'adoptovati',
 'adresovati',
 'agitovati',
 'agonizovati',
 'akcentovati',
 'akceptovati',
 'aklimatizovati',
 'akompanovati',
 'aktivovati',
 'aktualizovati',
 'akumulovati',
 'amnestovati',
 'amortizovati',
 'amputovati',
 'analizovati',
 'aneksovati',
 'anulovati',
 'apelovati',
 'aplodovati',
 'aranževati',
 'areštovati',
 'argumentovati',
 'arhivovati',
 'asimilovati',
 'atakovati',
 'avansovati',
 'avtomatizovati',
 'avtorizovati',
 'avtostopovati',
 'bagatelizovati',
 'balansovati',
 'balotovati',
 'balzamovati',
 'barikadovati',
 'barviti',
 'bazovati',
 '†pokojiti',
 'blågodariti',
 'blågoslavjati',
 'blågosloviti',
 'blågovolěti',
 'blågoželati',
 'blaznovati',
 'blědněti',
 'blěskati',
 'blěstěti',
 'bližiti',
 'bljunųti',
 'bljuvati',
 'blokovati',
 'blųkati',
 'bogatěti',
 'bogohuliti',
 'bojkotovati',
 'bombardovati',
 'brahtati',
 'bratati',
 'bråzditi',
 'breknųti',
 'brenča

In [748]:
morphemes.query('not ((isv in @pra_forms) or (base_verb.str.replace("†", "") in @pra_forms))')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
5,11.0,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,,,abdiko,vati,0,
16,6119.0,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,,,,abono,vati,0,
28,19625.0,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,absorbo,vati,0,
29,24056.0,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,abstraho,vati,0,
43,24070.0,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,,,adapto,vati,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17904,2627.0,žuvati,(žuje),v.intr. ipf.,"chew, masticate",NaN,verb,?,?,žuvati,žuvati,žuv,žuvati,,,,žu,vati,0,
17905,19473.0,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,,,,žuž,ati,0,
17913,-37005.0,råzpečętati,NaN,v.tr. pf.,unseal,NaN,verb,råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,,,råz,pečęt,ati,0,
17914,-37006.0,råzpečętyvati,NaN,v.tr. ipf.,unseal,NaN,verb,råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,,,råz,pečęty,vati,0,


In [720]:
morphemes.loc[
    morphemes.query('isv in @VALS').index,
    'reconstructed'
] = morphemes.query('isv in @VALS').isv

C:\Users\79165\AppData\Local\Temp\ipykernel_17612\2085135537.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  morphemes.loc[


In [721]:
morphemes

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
5,11.0,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,,,abdiko,vati,0,
16,6119.0,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,,,,abono,vati,0,
28,19625.0,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,absorbo,vati,0,
29,24056.0,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,abstraho,vati,0,
43,24070.0,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,,,adapto,vati,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473.0,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,,,,žuž,ati,0,
17913,-37005.0,råzpečętati,NaN,v.tr. pf.,unseal,NaN,verb,råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,,,råz,pečęt,ati,0,
17914,-37006.0,råzpečętyvati,NaN,v.tr. ipf.,unseal,NaN,verb,råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,,,råz,pečęty,vati,0,
17917,-30731.0,smogti,NaN,v.tr. pf.,"manage to, be able to, cope, make it",NaN,verb,?,?,mogti,smogti,smogti,mogti,,,s,mꜵg,ti,0,


In [935]:
prefix_map = {
    "nedoråz": "¬ne¬do¬råz",
    "nezado": "¬ne¬za¬do",
    # "poblågo":
    "prěvȯzȯ": "¬prě¬vȯz¬ȯ" 
}
weird_pref = set()


for p in morphemes._prefix.unique():
    if not p: continue
    if p in prefix_map: continue
    if p in possible_prefixes:
        prefix_map[p] = "¬" + p
    else:
        for length in range(1, len(p)):
            if p[:length] in possible_prefixes and p[length:] in possible_prefixes | {'ȯ', 'z'}:
                prefix_map[p] = "¬" + p[:length] + "¬" + p[length:]
                break
        else:
            print(p)
            weird_pref.add(p)
                
prefix_map


a
blågo
mråzo
poblågo
protiv
rųko
samou
sebe
spolu


{'nedoråz': '¬ne¬do¬råz',
 'nezado': '¬ne¬za¬do',
 'prěvȯzȯ': '¬prě¬vȯz¬ȯ',
 'bez': '¬bez',
 'de': '¬de',
 'do': '¬do',
 'doråz': '¬do¬råz',
 'doz': '¬do¬z',
 'iz': '¬iz',
 'izna': '¬iz¬na',
 'izne': '¬iz¬ne',
 'izȯ': '¬iz¬ȯ',
 'izu': '¬iz¬u',
 'na': '¬na',
 'nad': '¬nad',
 'nadu': '¬nad¬u',
 'napo': '¬na¬po',
 'nas': '¬na¬s',
 'po': '¬po',
 'sȯ': '¬sȯ',
 'ne': '¬ne',
 'nedo': '¬ne¬do',
 'nena': '¬ne¬na',
 'o': '¬o',
 'ob': '¬ob',
 'obez': '¬obez',
 'obȯ': '¬ob¬ȯ',
 'od': '¬od',
 'odȯ': '¬od¬ȯ',
 'odpo': '¬od¬po',
 'odza': '¬od¬za',
 'opo': '¬o¬po',
 'os': '¬o¬s',
 'pod': '¬pod',
 'podȯ': '¬pod¬ȯ',
 'podråz': '¬pod¬råz',
 'poråz': '¬po¬råz',
 'posȯ': '¬po¬sȯ',
 'pos': '¬po¬s',
 'pov': '¬po¬v',
 'poza': '¬po¬za',
 'poz': '¬po¬z',
 'prě': '¬prě',
 'prěd': '¬prěd',
 'prědȯ': '¬prěd¬ȯ',
 'prědpo': '¬prěd¬po',
 'prědpri': '¬prěd¬pri',
 'prěds': '¬prěd¬s',
 'prěna': '¬prě¬na',
 'prěo': '¬prě¬o',
 'prěpo': '¬prě¬po',
 'prěråz': '¬prě¬råz',
 'prěs': '¬prě¬s',
 'prěvȯz': '¬prě¬vȯz',
 'pri': '¬p

In [936]:
weird_pref


{'a', 'blågo', 'mråzo', 'poblågo', 'protiv', 'rųko', 'samou', 'sebe', 'spolu'}

In [923]:
for i, row in morphemes.query("genesis == 'I' and isv.str.startswith('a')").iterrows():
    if row.isv in 'akompanovati anulovati asimilovati'.split(" "):
        morphemes.loc[i, '_prefix'] = 'a'
        morphemes.loc[i, '_stem'] = morphemes.loc[i, '_stem'][1:]
    if row._stem.endswith("o"):
        morphemes.loc[i, '_stem'] = morphemes.loc[i, '_stem'][:-1]
        morphemes.loc[i, '_suffix'] = "o" + morphemes.loc[i, '_suffix']

morphemes.query("genesis == 'I' and isv.str.startswith('a')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
5,11.0,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,,,abdik,ovati,0,NaN
28,19625.0,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,absorb,ovati,0,NaN
29,24056.0,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,abstrah,ovati,0,NaN
43,24070.0,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,,,adapt,ovati,0,NaN
50,24081.0,administrovati,NaN,v.tr. ipf.,administrate,I,verb,?,?,administrovati,administrovati,administr,administrovati,|administracija,,,administr,ovati,0,NaN
54,19628.0,adoptovati,NaN,v.tr. ipf.,adopt,I,verb,?,?,adoptovati,adoptovati,adopt,adoptovati,,,,adopt,ovati,0,NaN
59,4540.0,adresovati,NaN,v.tr. ipf.,address,I,verb,?,?,adresovati,adresovati,adres,adresovati,,adres|adresa,,adres,ovati,0,NaN
92,24114.0,agitovati,NaN,v.intr. ipf.,agitate,I,verb,?,?,agitovati,agitovati,agit,agitovati,|agitacija|agitka,,,agit,ovati,0,NaN
99,24121.0,agonizovati,NaN,v.intr. ipf.,"agonise, agonize",I,verb,?,?,agonizovati,agonizovati,agoniz,agonizovati,,agonija,,agoniz,ovati,0,NaN
120,24148.0,akcentovati,NaN,v.tr. ipf.,"accentuate, stress (pronunciation)",I,verb,?,?,akcentovati,akcentovati,akcent,akcentovati,,akcent,,akcent,ovati,0,NaN


In [925]:
morphemes.genesis.value_counts()

I    335
F     11
D      8
E      6
S      2
M      1
Name: genesis, dtype: int64

In [929]:
for i, row in morphemes.query("genesis in ['I', 'F', 'D', 'E', 'S', 'M'] and _stem.str.endswith('o')").iterrows():
    morphemes.loc[i, '_stem'] = morphemes.loc[i, '_stem'][:-1]
    morphemes.loc[i, '_suffix'] = "o" + morphemes.loc[i, '_suffix']


In [930]:
morphemes.query("genesis in ['I', 'F', 'D', 'E', 'S', 'M'] and _stem.str.endswith('ir')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
2916,25578.0,garnirovati,NaN,v.tr. ipf.,garnish,F,verb,?,?,garnirovati,garnirovati,garnir,garnirovati,,,,garnir,ovati,0,NaN
3229,2668.0,gravirovati,NaN,v.tr. ipf.,engrave,I,verb,?,?,gravirovati,gravirovati,gravir,gravirovati,,,,gravir,ovati,0,NaN
3790,35896.0,inspirovati,NaN,v.tr. ipf./pf.,inspire,I,verb,?,?,inspirovati,inspirovati,inspir,inspirovati,|inspiracija,,,inspir,ovati,0,NaN
6333,2869.0,marširovati,NaN,v.intr. ipf.,march,D,verb,?,?,marširovati,marširovati,maršir,marširovati,,,,maršir,ovati,0,NaN
6342,33371.0,masirovati,NaN,v.tr. ipf.,massage,F,verb,?,?,masirovati,masirovati,masir,masirovati,,,,masir,ovati,0,NaN
10165,10565.0,pirovati,NaN,v.intr. ipf.,feast,I,verb,?,?,pirovati,pirovati,pir,pirovati,,pir,,pir,ovati,0,NaN
10657,10857.0,polirati,NaN,v.tr. ipf.,polish,I,verb,?,?,polirati,polirati,polir,?lirati?,,,,polir,ati,0,NaN
12640,36411.0,råzkvartirovati,NaN,v.tr. pf.,quarter (troops),I,verb,råz’,kvartirovati,råzkvartirovati,råzkvartirovati,råzkvartir,?kvartirovati?,,,,råzkvartir,ovati,0,NaN
12641,24162.0,råzkvartirovyvati,NaN,v.tr. ipf.,quarter (troops),I,verb,råz’,kvartirovyvati,råzkvartirovati,råzkvartirovyvati,råzkvartir,?kvartirovyvati?,,,,råzkvartir,ovyvati,0,NaN
13219,18648.0,rokirovati,NaN,v.intr. ipf./pf.,castle (chess),I,verb,?,?,rokirovati,rokirovati,rokir,rokirovati,,,,rokir,ovati,0,NaN


In [934]:
for i, row in morphemes.query("_stem.str.startswith('råz')").iterrows():
    morphemes.loc[i, '_prefix'] = 'råz'
    morphemes.loc[i, '_stem'] = morphemes.loc[i, '_stem'][3:]


In [921]:

morphemes.loc[13627, ["base_verb", '_prefix', "_stem"]] = ['sikati', '', 'sik']

morphemes.loc[3661, ["reconstructed", '_prefix', "_stem"]] = ['jьkati', '', 'ik']
morphemes.loc[17210, ["reconstructed", '_prefix', "_stem"]] = ['zajьkati', 'za', 'jik']



In [920]:
morphemes.query("en.str.contains('pee')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
6206,27352.0,lupiti,NaN,v.tr. ipf.,peel,NaN,verb,?,?,lupiti,lupiti,lup,lupiti,,,,lup,iti,0,lupiti
6761,1829.0,močiti,NaN,v.refl. ipf.,"urinate, pee",NaN,verb,?,?,močiti,močiti sę,moč,močiti,,moč|!moć,,moč,iti,0,močiti
7751,28421.0,něměti,NaN,v.intr. ipf.,"fall silent, become speechless, grow dumb, be ...",NaN,verb,?,?,něměti,něměti,něm,něměti,|němcija|němka|němec|němosť,,,něm,ěti,0,něměti
8280,28724.0,obdirati,NaN,v.tr. ipf.,"peel, skin, strip",NaN,verb,?,?,dreti,obdirati,obdir,?dirati?,,,ob,dir,ati,0,NaN
8284,34086.0,obdreti,(obdere),v.tr. pf.,"peel, skin, strip",NaN,verb,?,?,dreti,obdreti,obdreti,dreti,,,ob,dre,ti,0,NaN
8408,28822.0,oblupiti,NaN,v.tr. pf.,peel,NaN,verb,?,?,lupiti,oblupiti,oblup,lupiti,,,ob,lup,iti,0,(ob)+lupiti
9068,4304.0,odšlupati,NaN,v.tr. pf.,peel,NaN,verb,?,?,odšlupati,odšlupati,odšlup,?šlupati?,,,od,šlup,ati,0,NaN
9069,5552.0,odšlupyvati,NaN,v.tr. ipf.,peel,NaN,verb,?,?,odšlupati,odšlupyvati,odšlup,?šlupyvati?,,,od,šlup,yvati,0,NaN
9320,32831.0,oněměti,NaN,v.intr. pf.,"be struck dumb, fall silent, become speechless...",NaN,verb,?,?,něměti,oněměti,oněm,něměti,,,o,něm,ěti,0,(o)+něměti
10181,14838.0,pišati,NaN,v.intr. ipf.,pee,NaN,verb,?,?,pišati,pišati,piš,pišati,,,,piš,ati,0,NaN


In [916]:
morphemes.query("_prefix in @weird_pref")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
130,24156.0,aklimatizovati,NaN,v.intr. ipf.,acclimatize,I,verb,?,?,aklimatizovati,aklimatizovati,aklimatiz,aklimatizovati,|aklimatizacija,klimat,a,klimat,izovati,0,NaN
887,4889.0,blågodariti,NaN,v.tr. ipf.,thank,NaN,verb,?,?,dariti,blågodariti,blågodar,blågodariti,,,blågo,dar,iti,0,(blågo)+dariti
899,5574.0,blågoslavjati,NaN,v.tr. ipf.,"bless, beatify",NaN,verb,?,?,sloviti,blågoslavjati,blågoslav,blågoslavjati,,,blågo,slav,jati,0,?sloviti
900,5573.0,blågosloviti,NaN,v.tr. pf.,"bless, beatify",NaN,verb,?,?,sloviti,blågosloviti,blågosl,blågosloviti,,,blågo,slov,iti,0,(blågo)+sloviti
907,22297.0,blågovolěti,(blågovoli),v.intr. ipf.,"show benevolence, show goodwill",NaN,verb,?,?,volěti,blågovolěti,blågovol,blågovolěti,,,blågo,vol,ěti,0,NaN
912,6050.0,blågoželati,(+3),v.intr. ipf.,congratulate,NaN,verb,?,?,želati,blågoželati,blågožel,blågoželati,,,blågo,žel,ati,0,(blågo)+želati
6915,17812.0,mråzosušati,NaN,v.tr. ipf.,freeze-dry,NaN,verb,?,?,mråzosušati,mråzosušati,mråzosuš,mråzosušati,,,mråzo,suh,⒥ati,0,NaN
6918,9263.0,mråzosušiti,NaN,v.tr. pf.,freeze-dry,NaN,verb,?,?,mråzosušati,mråzosušiti,mråzosuš,mråzosušiti,,,mråzo,suh,⒥iti,0,NaN
7668,20535.0,nedoråzuměti,NaN,v.tr. pf.,misunderstand,NaN,verb,?,?,uměti,nedoråzuměti,nedoråzum,uměti,,,nedoråz,um,ěti,0,?orzuměti
8064,17948.0,nezadovaljati,NaN,v.tr. ipf.,"displease, dissatisfy",NaN,verb,?,?,voliti,nezadovaljati,nezadoval,valjati,,,nezado,vꜵl,jati,0,?voliti


In [938]:
morphemes._prefix = morphemes._prefix.apply(lambda x: prefix_map.get(x, x))

In [939]:
morphemes.to_pickle("morphemes_protoslavic.pkl")
morphemes[['id', 'isv_orig', 'addition', 'partOfSpeech', 'en', 'genesis', 
          'isv', 'base_verb', 'derived_nouns', 'base_noun', '_prefix', '_stem',
       '_suffix', 'reconstructed']].to_csv("morphemes_protoslavic.csv")

In [948]:
print(
    morphemes.query("_prefix.str.contains('¬')")._prefix.value_counts().to_csv(sep="\t")
)

	_prefix
¬råz	349
¬iz	261
¬od	253
¬na	242
¬o	242
¬za	208
¬po	186
¬u	174
¬ob	152
¬s	149
¬prě	117
¬pri	114
¬pro	89
¬do	87
¬v	65
¬pod	52
¬vȯz	44
¬sȯ	25
¬prěd	22
¬sų	12
¬råz¬pro	9
¬obez	7
¬vy	7
¬s¬po	7
¬råz¬s	6
¬u¬s	5
¬nad	5
¬prě¬s	5
¬za¬ne	5
¬de	4
¬prě¬o	4
¬pro¬iz	4
¬po¬z	4
¬po¬v	4
¬pri¬po	4
¬o¬s	4
¬za¬do	4
¬ob¬ȯ	4
¬iz¬na	4
¬vȯ	4
¬prěd¬po	3
¬na¬s	3
¬po¬za	3
¬od¬po	3
¬ne	3
¬vȯz¬po	2
¬pro¬ne	2
¬s¬na	2
¬råz¬o	2
¬råz¬råz	2
¬råz¬po	2
¬sų¬pro	2
¬s¬ne	2
¬bez	2
¬prěd¬pri	2
¬za¬u	2
¬do¬z	2
¬ne¬za¬do	2
¬nad¬u	2
¬iz¬u	2
¬pod¬ȯ	2
¬pod¬råz	2
¬po¬råz	2
¬na¬po	2
¬prěd¬s	2
¬prě¬na	2
¬prě¬po	2
¬prě¬råz	2
¬do¬råz	1
¬ne¬do¬råz	1
¬za¬po	1
¬v¬z	1
¬iz¬ȯ	1
¬vȯz¬ȯ	1
¬za¬pro	1
¬iz¬ne	1
¬pri¬na	1
¬ne¬do	1
¬ne¬na	1
¬od¬ȯ	1
¬prě¬vȯz¬ȯ	1
¬od¬za	1
¬o¬po	1
¬s¬de	1
¬po¬sȯ	1
¬po¬s	1
¬prěd¬ȯ	1
¬råz¬ȯ	1
¬prě¬vȯz	1
¬s¬råz	1



In [949]:
morphemes.query("en.str.contains('dis')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
785,19654.0,bezpokojiti,NaN,v.tr. ipf.,"disturb, trouble, upset, bother",NaN,verb,?,?,†pokojiti,bezpokojiti,bezpoko,?kojiti?,,pokoj,¬bez,pokoj,iti,0,NaN
1875,25241.0,dezinfikovati,NaN,v.tr. ipf./pf.,disinfect,I,verb,dez’,infikovati,dezinfikovati,dezinfikovati,dezinfik,?zinfikovati?,,,,dezinfik,ovati,0,NaN
1936,35510.0,diskreditovati,NaN,v.tr. ipf./pf.,discredit,I,verb,?,?,diskreditovati,diskreditovati,diskredit,diskreditovati,,,,diskredit,ovati,0,NaN
1939,32603.0,diskriminovati,NaN,v.intr. ipf./pf.,discriminate,I,verb,?,?,diskriminovati,diskriminovati,diskrimin,diskriminovati,|diskriminacija,,,diskrimin,ovati,0,NaN
1942,301.0,diskutovati,NaN,v.intr. ipf.,discuss,I,verb,?,?,diskutovati,diskutovati,diskut,diskutovati,,,,diskut,ovati,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16006,3558.0,uvoljniti,NaN,v.tr. pf.,"discharge, dismiss, fire (terminate employment)",NaN,verb,?,?,voliti,uvoljniti,uvoljn,?ljniti?,,,¬u,vꜵl,jniti,0,?voliti
17176,5475.0,zahoditi,NaN,v.intr. ipf.,set (disappear behind the horizon),NaN,verb,?,?,hoditi,zahoditi,zahod,hoditi,,,¬za,hꜵd,iti,0,(za)+xoditi
17194,5476.0,zajdti,(zajde; zašel),v.intr. pf.,set (disappear behind the horizon),NaN,verb,?,?,idti,zajdti,zajdti,?jdti?,,,¬za,jd,ti,0,NaN
17407,21238.0,zapropastiti,NaN,v.refl. pf.,disappear completely,NaN,verb,?,?,pasti,zapropastiti sę,zapropast,?pastiti?,,,¬za¬pro,pas,titi,0,?pasti


In [951]:
morphemes.query("isv.str.contains('^d[ie][zs]')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
1826,35733.0,destabilizovati,NaN,v.tr. ipf./pf.,destabilize,I,verb,de’,stabilizovati,stabilizovati,destabilizovati,destabiliz,stabilizovati,|destabilizacija,,¬de,stabiliz,ovati,0,NaN
1873,25239.0,dezertovati,NaN,v.intr. ipf./pf.,desert,I,verb,’,dezertovati,dezertovati,dezertovati,dezert,?zertovati?,,,,dezert,ovati,0,NaN
1875,25241.0,dezinfikovati,NaN,v.tr. ipf./pf.,disinfect,I,verb,dez’,infikovati,dezinfikovati,dezinfikovati,dezinfik,?zinfikovati?,,,,dezinfik,ovati,0,NaN
1936,35510.0,diskreditovati,NaN,v.tr. ipf./pf.,discredit,I,verb,?,?,diskreditovati,diskreditovati,diskredit,diskreditovati,,,,diskredit,ovati,0,NaN
1939,32603.0,diskriminovati,NaN,v.intr. ipf./pf.,discriminate,I,verb,?,?,diskriminovati,diskriminovati,diskrimin,diskriminovati,|diskriminacija,,,diskrimin,ovati,0,NaN
1942,301.0,diskutovati,NaN,v.intr. ipf.,discuss,I,verb,?,?,diskutovati,diskutovati,diskut,diskutovati,,,,diskut,ovati,0,NaN
1944,35295.0,diskvalifikovati,NaN,v.tr. ipf./pf.,disqualify,I,verb,?,?,diskvalifikovati,diskvalifikovati,diskvalifik,diskvalifikovati,|diskvalifikacija,,,diskvalifik,ovati,0,NaN
1946,35944.0,distancevati,NaN,v.refl. ipf./pf.,distance oneself,I,verb,?,?,distancevati,distancevati sę,distancev,distancevati,,,,distanc,evati,0,NaN


In [952]:
morphemes.query("base_verb.str.contains('dti')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
2057,5345.0,dojdti,(dojde; došel),v.intr. pf.,arrive,NaN,verb,?,?,idti,dojdti,dojdti,?jdti?,,,¬do,jd,ti,0,NaN
3638,13823.0,idti vprěd,#(ide; šel),v.ipf.,advance,NaN,verb,?,?,idti,idti vprěd,idti vprěd,?idti vprěd?,,,,jd,ti,0,NaN
3639,15654.0,idti vzad,#(ide; šel),v.ipf.,back up,NaN,verb,?,?,idti,idti vzad,idti vzad,?idti vzad?,,,,jd,ti,0,NaN
3640,2104.0,idti,(ide; šel),v.intr. ipf.,go,NaN,verb,?,?,idti,idti,idti,idti,,,,jd,ti,0,NaN
4124,36803.0,iznajdti,(iznajde),v.tr. pf.,invent (create something new),NaN,verb,?,?,idti,iznajdti,iznajdti,najdti,,,¬iz¬na,jd,ti,0,NaN
4148,5354.0,izȯjdti,(izȯjde; izšel),v.intr. pf.,"exit, go out, quit",NaN,verb,?,?,idti,izȯjdti,izȯjdti,?ȯjdti?,,,¬iz¬ȯ,jd,ti,0,NaN
7187,781.0,najdti,(najde; našel),v.tr. pf.,find,NaN,verb,?,?,idti,najdti,najdti,?dti?,,,¬na,jd,ti,0,NaN
8469,18025.0,obȯjdti,(obȯjde; obšel),v.refl. pf.,do without,NaN,verb,?,?,idti,obȯjdti sę bez,obȯjdti,?ȯjdti?,,,¬ob¬ȯ,jd,ti,0,NaN
8470,18023.0,obȯjdti,(obȯjde; obšel),v.refl. pf.,"manage, make do with, get by",NaN,verb,?,?,idti,obȯjdti sę,obȯjdti,?ȯjdti?,,,¬ob¬ȯ,jd,ti,0,NaN
8471,5370.0,obȯjdti,(obȯjde; obšel),v.tr. pf.,"go around, circumvent",NaN,verb,?,?,idti,obȯjdti,obȯjdti,?ȯjdti?,,,¬ob¬ȯ,jd,ti,0,NaN


In [953]:
morphemes.query("isv.str.contains('odryvati')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
9006,29227.0,odryvati,NaN,v.tr. ipf.,dig up,NaN,verb,?,?,ryti,odryvati,odr,?ryvati?,,,¬od,ry,vati,0,?ryti
9007,34361.0,odryvati,NaN,v.tr. ipf.,tear away,NaN,verb,?,?,rvati,odryvati,odr,?ryvati?,,,¬od,ryv,ati,0,?rъvati


In [958]:
morphemes.query("en.str.contains('separate')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
1781,1754.0,děliti,NaN,v.tr. ipf.,"share, divide, part, separate",NaN,verb,?,?,děliti,děliti,děl,děliti,,děl,,děl,iti,0,děliti
8756,2951.0,odděliti,NaN,v.tr. pf.,separate,NaN,verb,?,?,děliti,odděliti,odděl,děliti,,,¬od,děl,iti,0,(od)+děliti
8757,3016.0,odděljati,NaN,v.tr. ipf.,separate,NaN,verb,?,?,děliti,odděljati,odděl,?děljati?,,,¬od,děl,jati,0,?děliti
12508,595.0,råzděliti,NaN,v.tr. pf.,"separate, sever, divide",NaN,verb,råz’,děliti,děliti,råzděliti,råzděl,děliti,,,¬råz,děl,iti,0,(råz)+děliti
12509,2249.0,råzděljati,NaN,v.tr. ipf.,"separate, sever, divide",NaN,verb,råz’,děljati,děliti,råzděljati,råzděl,?děljati?,,,¬råz,děl,jati,0,?děliti
12676,20108.0,råzlųčati,NaN,v.tr. ipf.,separate,NaN,verb,råz’,lųčati,råzlųčati,råzlųčati,råzlųč,?lųčati?,|råzlųčnik,,¬råz,lųč,ati,0,NaN
12678,20109.0,råzlųčiti,NaN,v.tr. pf.,separate,NaN,verb,råz’,lųčiti,råzlųčati,råzlųčiti,råzlųč,?lųčiti?,|råzlųčnik,,¬råz,lųč,iti,0,NaN


In [960]:
morphemes.query("_prefix == '' and prefix != '?'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
1710,25144.0,debatovati,NaN,v.intr. ipf.,debate,I,verb,’,debatovati,,debatovati,debat,?batovati?,,debata,,debat,ovati,0,?uměti
1711,25146.0,debelěti,NaN,v.intr. ipf.,grow fat,NaN,verb,’,debelěti,belěti,debelěti,debel,belěti,,,,debel,ěti,0,NaN
1716,25152.0,debjutovati,NaN,v.intr. ipf./pf.,make one’s debut,F,verb,’,debjutovati,,debjutovati,debjut,?bjutovati?,,debjut,,debjut,ovati,0,?uměti
1729,4956.0,definiovati,NaN,v.tr. ipf.,define,I,verb,’,definiovati,definiovati,definiovati,defini,?finiovati?,|definicija,,,defini,ovati,0,NaN
1735,15280.0,defisovati,NaN,v.tr. ipf.,hyphenate,I,verb,’,defisovati,,defisovati,defis,?fisovati?,,defis,,defis,ovati,0,?uměti
1742,25172.0,degradovati,NaN,v.tr. ipf./pf.,"degrade, demote",I,verb,’,degradovati,degradovati,degradovati,degrad,?gradovati?,|degradacija,,,degrad,ovati,0,NaN
1744,25175.0,degustovati,NaN,v.tr. ipf./pf.,taste,I,verb,’,degustovati,degustovati,degustovati,degust,?gustovati?,|degustacija,,,degust,ovati,0,NaN
1757,35941.0,deklamovati,NaN,v.tr. ipf./pf.,"declaim, recite",I,verb,’,deklamovati,deklamovati,deklamovati,deklam,?klamovati?,,,,deklam,ovati,0,NaN
1795,6266.0,demonstrovati,NaN,v.tr. ipf.,demonstrate,I,verb,’,demonstrovati,demonstrovati,demonstrovati,demonstr,?monstrovati?,|demonstracija,,,demonstr,ovati,0,NaN
1806,398.0,deportovati,NaN,v.tr. ipf./pf.,deport,I,verb,de’,portovati,deportovati,deportovati,deport,?rtovati?,|deportacija,,,deport,ovati,0,NaN


In [990]:

morphemes.query("_prefix == '' and prefix not in ['’', '?']")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
1806,398.0,deportovati,NaN,v.tr. ipf./pf.,deport,I,verb,de’,portovati,deportovati,deportovati,deport,?rtovati?,|deportacija,,,deport,ovati,0,NaN
1875,25241.0,dezinfikovati,NaN,v.tr. ipf./pf.,disinfect,I,verb,dez’,infikovati,dezinfikovati,dezinfikovati,dezinfik,?zinfikovati?,,,,dezinfik,ovati,0,NaN
8298,773.0,obezglåviti,NaN,v.tr. pf.,decapitate,NaN,verb,obez’,glåviti,obezglåviti,obezglåviti,obezglåv,?glåviti?,,,,obezglåv,iti,0,NaN
8299,1587.0,obezglåvjati,NaN,v.tr. ipf.,decapitate,NaN,verb,obez’,glåvjati,obezglåviti,obezglåvjati,obezglåv,?glåvjati?,,,,obezglåv,jati,0,NaN
8303,5374.0,obezsiliti,NaN,v.tr. pf.,disempower,NaN,verb,obez’,siliti,obezsiliti,obezsiliti,obezsil,siliti,,,,obezsil,iti,0,NaN
8304,5373.0,obezsiljati,NaN,v.tr. ipf.,disempower,NaN,verb,obez’,siljati,obezsiliti,obezsiljati,obezsil,?iljati?,,,,obezsil,jati,0,NaN
14056,34940.0,sȯčuvstvovati,NaN,v.intr. ipf.,"sympathize, commiserate, feel compassion",NaN,verb,sȯ’,čuvstvovati,čuvati,sȯčuvstvovati,sȯčuvstv,?čuvstvovati?,,,,,,1,?čuti
14076,34119.0,sȯhnųti,NaN,v.intr. ipf.,"dry (intr.), become dry, wither",NaN,verb,sȯ’,hnųti,sȯhnųti,sȯhnųti,sȯh,?hnųti?,,,,sȯh,nųti,0,NaN
14077,34121.0,sȯhnųti,NaN,v.intr. ipf.,pine away,NaN,verb,sȯ’,hnųti,sȯhnųti,sȯhnųti,sȯh,?hnųti?,,,,sȯh,nųti,0,NaN
14119,4150.0,sȯsati,NaN,v.tr. ipf.,suck,NaN,verb,sȯ’,sati,sȯsati,sȯsati,sȯs,?ati?,|sȯska,,,sȯs,ati,0,NaN


In [978]:
morphemes.query("_prefix != '' and reconstructed == reconstructed")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
887,4889.0,blågodariti,NaN,v.tr. ipf.,thank,NaN,verb,?,?,dariti,blågodariti,blågodar,blågodariti,,,blågo,dar,iti,0,(blågo)+dariti
899,5574.0,blågoslavjati,NaN,v.tr. ipf.,"bless, beatify",NaN,verb,?,?,sloviti,blågoslavjati,blågoslav,blågoslavjati,,,blågo,slav,jati,0,?sloviti
900,5573.0,blågosloviti,NaN,v.tr. pf.,"bless, beatify",NaN,verb,?,?,sloviti,blågosloviti,blågosl,blågosloviti,,,blågo,slov,iti,0,(blågo)+sloviti
912,6050.0,blågoželati,(+3),v.intr. ipf.,congratulate,NaN,verb,?,?,želati,blågoželati,blågožel,blågoželati,,,blågo,žel,ati,0,(blågo)+želati
1983,21272.0,dobaviti,NaN,v.tr. pf.,add,NaN,verb,?,?,baviti,dobaviti,dobav,baviti,,,¬do,bav,iti,-1,(do)+baviti
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17580,32644.0,zažegti,(zažže),v.tr. pf.,set fire to,NaN,verb,?,?,žegti,zažegti,zažegti,žegti,,,¬za,žeg,ti,0,(za)+žeťi
17581,32633.0,zažigati,NaN,v.tr. ipf.,"ignite, light, kindle, inflame",NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,¬za,žig,ati,0,?žeťi
17582,32636.0,zažigati,NaN,v.tr. ipf.,"switch on (the lights), turn on (the light)",NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,¬za,žig,ati,0,?žeťi
17583,32638.0,zažigati,NaN,v.tr. ipf.,set fire to,NaN,verb,?,?,žegti,zažigati,zažig,?žigati?,,,¬za,žig,ati,0,?žeťi


In [987]:

def bite_all_prefixes_off(word, verb_nest):

    can_continue = True
    prefixes = []

    while can_continue:
        can_continue = False
        for pref in possible_prefixes:
            if word.startswith(pref):
                # print(pref, word, word[len(pref):])
                new_word = word[len(pref):]
                if new_word.endswith(verb_nest):
                    word = new_word
                    prefixes.append(pref)
                    can_continue = True
                    break
    return ("¬" if prefixes else "") + "¬".join(prefixes)

In [989]:
morphemes.query("_prefix == ''").isv.apply(lambda x: bite_all_prefixes_off(x, "")).value_counts().head(33)

          1054
¬s         137
¬v          99
¬o          58
¬ob         55
¬po         48
¬u          45
¬za         22
¬od         22
¬iz         20
¬o¬s        18
¬pro        14
¬pri        12
¬po¬s       12
¬s¬v        11
¬de         11
¬prě        11
¬za¬s       10
¬u¬s        10
¬prě¬s       8
¬iz¬s        8
¬do¬s        7
¬na          7
¬v¬o         7
¬iz¬v        7
¬pri¬s       6
¬pro¬v       6
¬o¬v         6
¬o¬po        6
¬u¬po        6
¬sȯ¬v        5
¬sȯ          5
¬s¬po        5
Name: isv, dtype: int64

In [982]:
morphemes[morphemes._stem.str.len() > 5]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,reconstructed
28,19625.0,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,absorb,ovati,0,NaN
29,24056.0,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,abstrah,ovati,0,NaN
50,24081.0,administrovati,NaN,v.tr. ipf.,administrate,I,verb,?,?,administrovati,administrovati,administr,administrovati,|administracija,,,administr,ovati,0,NaN
99,24121.0,agonizovati,NaN,v.intr. ipf.,"agonise, agonize",I,verb,?,?,agonizovati,agonizovati,agoniz,agonizovati,,agonija,,agoniz,ovati,0,NaN
120,24148.0,akcentovati,NaN,v.tr. ipf.,"accentuate, stress (pronunciation)",I,verb,?,?,akcentovati,akcentovati,akcent,akcentovati,,akcent,,akcent,ovati,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17673,1892.0,zloupotrěbjati,NaN,v.tr. ipf.,"abuse, misuse",NaN,verb,?,?,zloupotrěbiti,zloupotrěbjati,zloupotrěb,zloupotrěbjati,,,,zloupotrěb,jati,0,NaN
17712,22850.0,znamenovati,NaN,v.tr. ipf.,"mark, put a mark on",NaN,verb,?,?,znamenovati,znamenovati,znamen,znamenovati,|znameńje|znameńje,,,znameno,vati,0,NaN
17713,22851.0,znamenovati,NaN,v.tr. ipf.,"signify, mean, be a sign of",NaN,verb,?,?,znamenovati,znamenovati,znamen,znamenovati,|znameńje|znameńje,,,znameno,vati,0,NaN
17914,-37006.0,råzpečętyvati,NaN,v.tr. ipf.,unseal,NaN,verb,råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,,,¬råz,pečęty,vati,0,NaN


In [964]:

morphemes.query("_prefix == ''").isv.str[:3].value_counts()

prě    33
pri    29
pro    25
obr    22
str    18
       ..
let     1
lep     1
lěn     1
lat     1
žuž     1
Name: isv, Length: 869, dtype: int64

In [973]:
morphemes.query("_prefix == '' and right_stem_cand != isv and not isv.str.contains(' ') and not right_stem_cand.str.startswith('?')").base_verb.value_counts()

vvŕgati        4
dostigati      4
zaustaviti     4
postaviti      4
podključati    4
              ..
podslušati     1
poučati        1
povědati       1
prěslušati     1
zavojevati     1
Name: base_verb, Length: 77, dtype: int64